# Mining Input Grammar from Binaries

* Code for subjects [here](#Our-subject-programs)
* Evaluation starts [here](#Evaluation)
  * The evaluation on specific subjects starts [here](#Subjects)
    * [CGIDecode](#CGIDecode)
    * [Calculator](#Calculator)
    * [CSVParser](#CSV)
    * [URLParse](#URLParse)
    * [Json](#Json)
    * [INI](#INI)
* Results are [here](#Results)

## Install Prerequisites

In [ ]:
import fuzzingbook_utils

## Our subject programs

In [ ]:
!mkdir subjects

### Calculator.c

This is a really simple calculator written in text book recursive descent style.

In [ ]:
calc_parse = """\
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <ctype.h>
#include "stdbool.h"
#define MAX_STR_SIZE 10000

typedef struct str {
  char my_string[MAX_STR_SIZE];
  int idx;
} my_arg;

void parse_num(my_arg *arg);
void parse_expr(my_arg *arg);
void parse_paren(my_arg *arg);

int nesting = 0;

void parse_num(my_arg *arg) {
    for (;arg->idx < strlen(arg->my_string); arg->idx++) {
        if (!isdigit(arg->my_string[arg->idx])) {
            break;
        }
    }
    return;
}

void parse_paren(my_arg *arg) {
  if (arg->my_string[arg->idx] != '(') {
    return;
  }

  arg->idx += 1;
  parse_expr(arg);
  if (arg->my_string[arg->idx] != ')') {
    return;
  }
  arg->idx += 1;
}

void parse_expr(my_arg *arg) {
  while (arg->idx < strlen(arg->my_string)) {
    char c = arg->my_string[arg->idx];
    if (isdigit(c)) {
      parse_num(arg);
    } else if (c == '+' || c == '-' || c == '*' || c == '/') {
      arg->idx += 1;
    } else if (c == '(') {
      parse_paren(arg);
    } else if (c == ')') {
      break;
    }
  }
}

int main(int argc, char *argv[]) {
    my_arg arg;
    strcpy(arg.my_string, argv[1]);
    arg.idx = 0;
    parse_expr(&arg);
}
"""

In [ ]:
with open('subjects/calc_parse.c', 'w+') as f:
    print(calc_parse, file=f)

### CGIDecode.c

The CGIDecode is a program to decode a URL encoded string. The source for this program was obtained from [here](https://www.fuzzingbook.org/html/Coverage.html).

In [ ]:
cgi_decode = """\
#include <stdio.h>
#include <stdlib.h>
#include <string.h>


int hex_values[256];


int cgi_decode(char *s, char *t) {
  for (int i = 0; i < sizeof(hex_values) / sizeof(int); i++) {
  hex_values[i] = -1;
  }
  hex_values['0'] = 0;
  hex_values['1'] = 1;
  hex_values['2'] = 2;
  hex_values['3'] = 3;
  hex_values['4'] = 4;
  hex_values['5'] = 5;
  hex_values['6'] = 6;
  hex_values['7'] = 7;
  hex_values['8'] = 8;
  hex_values['9'] = 9;

  hex_values['a'] = 10;
  hex_values['b'] = 11;
  hex_values['c'] = 12;
  hex_values['d'] = 13;
  hex_values['e'] = 14;
  hex_values['f'] = 15;

  hex_values['A'] = 10;
  hex_values['B'] = 11;
  hex_values['C'] = 12;
  hex_values['D'] = 13;
  hex_values['E'] = 14;
  hex_values['F'] = 15;

  while (*s != '\\0') {
    if (*s == '+') {
      *t++ = ' ';
    } else if (*s == '%') {
      int digit_high = *++s;
      int digit_low = *++s;
      if (hex_values[digit_high] >= 0 && hex_values[digit_low] >= 0) {
        *t++ = hex_values[digit_high] * 16 + hex_values[digit_low];
      } else {
        return -1;
      }
    } else {
      *t++ = *s;
    }
    s++;
  }
  *t = '\\0';
  return 0;
}

int main(int argc, char *argv[]) {
  char my_string[10240];
  char result[10240];
  int ret = -1;
  
  strcpy(my_string, argv[1]);
  ret = cgi_decode(my_string, result);
  return ret;
}
"""

In [ ]:
with open('subjects/cgi_decode.c', 'w+') as f:
    print(cgi_decode, file=f)

### URLParse.c

In [ ]:
url_parse = """\
/*
 * urlparse.c
 *
 * Decompose a URL into its components.
 */
#include <string.h>
#include <stdlib.h>
#include <stdio.h>

enum url_type {
    URL_NORMAL,
    URL_OLD_TFTP,
    URL_PREFIX
};

struct url_info {
    char *scheme;
    char *user;
    char *passwd;
    char *host;
    unsigned int port;
    char *path;			/* Includes query */
    enum url_type type;
};

void parse_url(struct url_info *ui, char *url){
    char *p = url;
    char *q, *r, *s;

    memset(ui, 0, sizeof *ui);

    q = strstr(p, "://");
    if (!q) {
        q = strstr(p, "::");
        if (q) {
            *q = '\\000';
            ui->scheme = "tftp";
            ui->host = p;
            ui->path = q+2;
            ui->type = URL_OLD_TFTP;
            return;
        } else {
            ui->path = p;
            ui->type = URL_PREFIX;
            return;
        }
    }

    ui->type = URL_NORMAL;

    ui->scheme = p;
    *q = '\\000';
    p = q+3;

    q = strchr(p, '/');
    if (q) {
        *q = '\\000';
        ui->path = q+1;
        q = strchr(q+1, '#');
    if (q)
        *q = '\\000';
    } else {
        ui->path = "";
    }

    r = strchr(p, '@');
    if (r) {
        ui->user = p;
        *r = '\\000';
        s = strchr(p, ':');
        if (s) {
            *s = '\\000';
            ui->passwd = s+1;
        }
        p = r+1;
    }

    ui->host = p;
    r = strchr(p, ':');
    if (r) {
        *r = '\\000';
        ui->port = atoi(r+1);
    }
}

char *url_escape_unsafe(const char *input){
    const char *p = input;
    unsigned char c;
    char *out, *q;
    int n = 0;

    while ((c = *p++)) {
        if (c < ' ' || c > '~') {
            n += 3;		/* Need escaping */
        } else {
            n++;
        }
    }

    q = out = malloc(n+1);
    while ((c = *p++)) {
        if (c < ' ' || c > '~') {
            q += snprintf(q, 3, "%02X", c);
        } else {
            *q++ = c;
        }
    }

    *q = '\\000';

    return out;
}

static int hexdigit(char c){
    if (c >= '0' && c <= '9')
        return c - '0';
    c |= 0x20;
    if (c >= 'a' && c <= 'f')
        return c - 'a' + 10;
    return -1;
}

void url_unescape(char *buffer){
    const char *p = buffer;
    char *q = buffer;
    unsigned char c;
    int x, y;

    while ((c = *p++)) {
        if (c == '%') {
            x = hexdigit(p[0]);
            if (x >= 0) {
                y = hexdigit(p[1]);
                if (y >= 0) {
                    *q++ = (x << 4) + y;
                    p += 2;
                    continue;
                }
            }
        }
        *q++ = c;
    }
    *q = '\\000';
}


int main(int argc, char* argv[]) {
    struct url_info url;
    parse_url(&url, argv[1]);
    return 0;
}
"""

In [ ]:
with open('subjects/url_parse.c', 'w+') as f:
    print(url_parse, file=f)

### CSVParser.c

In [ ]:
csv_parse = """\
#include <stdlib.h>
#include <string.h>
#include <stdio.h>
#include <errno.h>

#ifndef CSVPARSER_H
#define CSVPARSER_H

#ifdef __cplusplus
extern "C" {
#endif

typedef struct CsvRow {
    char **fields_;
    int numOfFields_;
} CsvRow;

typedef struct CsvParser {
    char *filePath_;
    char delimiter_;
    int firstLineIsHeader_;
    char *errMsg_;
    CsvRow *header_;
    FILE *fileHandler_;
	int fromString_;
	char *csvString_;
	int csvStringIter_;
} CsvParser;


// Public
CsvParser *CsvParser_new(const char *filePath, const char *delimiter, int firstLineIsHeader);
CsvParser *CsvParser_new_from_string(const char *csvString, const char *delimiter, int firstLineIsHeader);
void CsvParser_destroy(CsvParser *csvParser);
void CsvParser_destroy_row(CsvRow *csvRow);
CsvRow *CsvParser_getHeader(CsvParser *csvParser);
CsvRow *CsvParser_getRow(CsvParser *csvParser);
int CsvParser_getNumFields(CsvRow *csvRow);
char **CsvParser_getFields(CsvRow *csvRow);
const char* CsvParser_getErrorMessage(CsvParser *csvParser);

// Private
CsvRow *_CsvParser_getRow(CsvParser *csvParser);    
int _CsvParser_delimiterIsAccepted(const char *delimiter);
void _CsvParser_setErrorMessage(CsvParser *csvParser, const char *errorMessage);

#ifdef __cplusplus
}
#endif

#endif

CsvParser *CsvParser_new(const char *filePath, const char *delimiter, int firstLineIsHeader) {
    CsvParser *csvParser = (CsvParser*)malloc(sizeof(CsvParser));
    if (filePath == NULL) {
        csvParser->filePath_ = NULL;
    } else {
        int filePathLen = strlen(filePath);
        csvParser->filePath_ = (char*)malloc((filePathLen + 1));
        strcpy(csvParser->filePath_, filePath);
    }
    csvParser->firstLineIsHeader_ = firstLineIsHeader;
    csvParser->errMsg_ = NULL;
    if (delimiter == NULL) {
        csvParser->delimiter_ = ',';
    } else if (_CsvParser_delimiterIsAccepted(delimiter)) {
        csvParser->delimiter_ = *delimiter;
    } else {
        csvParser->delimiter_ = '\\0';
    }
    csvParser->header_ = NULL;
    csvParser->fileHandler_ = NULL;
	csvParser->fromString_ = 0;
	csvParser->csvString_ = NULL;
	csvParser->csvStringIter_ = 0;

    return csvParser;
}

CsvParser *CsvParser_new_from_string(const char *csvString, const char *delimiter, int firstLineIsHeader) {
	CsvParser *csvParser = CsvParser_new(NULL, delimiter, firstLineIsHeader);
	csvParser->fromString_ = 1;	
	if (csvString != NULL) {
		int csvStringLen = strlen(csvString);
		csvParser->csvString_ = (char*)malloc(csvStringLen + 1);
		strcpy(csvParser->csvString_, csvString);
	}	
	return csvParser;
}

void CsvParser_destroy(CsvParser *csvParser) {
    if (csvParser == NULL) {
        return;
    }
    if (csvParser->filePath_ != NULL) {
        free(csvParser->filePath_);
    }
    if (csvParser->errMsg_ != NULL) {
        free(csvParser->errMsg_);
    }
    if (csvParser->fileHandler_ != NULL) {
        fclose(csvParser->fileHandler_);
    }
    if (csvParser->header_ != NULL) {
        CsvParser_destroy_row(csvParser->header_);
    }
	if (csvParser->csvString_ != NULL) {
		free(csvParser->csvString_);
	}
    free(csvParser);
}

void CsvParser_destroy_row(CsvRow *csvRow) {
    int i;
    for (i = 0 ; i < csvRow->numOfFields_ ; i++) {
        free(csvRow->fields_[i]);
    }
    free(csvRow);
}

CsvRow *CsvParser_getHeader(CsvParser *csvParser) {
    if (! csvParser->firstLineIsHeader_) {
        _CsvParser_setErrorMessage(csvParser, "Cannot supply header, as current CsvParser object does not support header");
        return NULL;
    }
    if (csvParser->header_ == NULL) {
        csvParser->header_ = _CsvParser_getRow(csvParser);
    }
    return csvParser->header_;
}

CsvRow *CsvParser_getRow(CsvParser *csvParser) {
    if (csvParser->firstLineIsHeader_ && csvParser->header_ == NULL) {
        csvParser->header_ = _CsvParser_getRow(csvParser);
    }
    return _CsvParser_getRow(csvParser);
}

int CsvParser_getNumFields(CsvRow *csvRow) {
    return csvRow->numOfFields_;
}

char **CsvParser_getFields(CsvRow *csvRow) {
    return csvRow->fields_;
}

CsvRow *_CsvParser_getRow(CsvParser *csvParser) {
    int numRowRealloc = 0;
    int acceptedFields = 64;
    int acceptedCharsInField = 64;
    if (csvParser->filePath_ == NULL && (! csvParser->fromString_)) {
        _CsvParser_setErrorMessage(csvParser, "Supplied CSV file path is NULL");
        return NULL;
    }
    if (csvParser->csvString_ == NULL && csvParser->fromString_) {
        _CsvParser_setErrorMessage(csvParser, "Supplied CSV string is NULL");
        return NULL;
    }
    if (csvParser->delimiter_ == '\\0') {
        _CsvParser_setErrorMessage(csvParser, "Supplied delimiter is not supported");
        return NULL;
    }
    if (! csvParser->fromString_) {
        if (csvParser->fileHandler_ == NULL) {
            csvParser->fileHandler_ = fopen(csvParser->filePath_, "r");
            if (csvParser->fileHandler_ == NULL) {
                int errorNum = errno;
                const char *errStr = strerror(errorNum);
                char *errMsg = (char*)malloc(1024 + strlen(errStr));
                strcpy(errMsg, "");
                sprintf(errMsg, "Error opening CSV file for reading: %s : %s", csvParser->filePath_, errStr);
                _CsvParser_setErrorMessage(csvParser, errMsg);
                free(errMsg);
                return NULL;
            }
        }
    }
    CsvRow *csvRow = (CsvRow*)malloc(sizeof(CsvRow));
    csvRow->fields_ = (char**)malloc(acceptedFields * sizeof(char*));
    csvRow->numOfFields_ = 0;
    int fieldIter = 0;
    char *currField = (char*)malloc(acceptedCharsInField);
    int inside_complex_field = 0;
    int currFieldCharIter = 0;
    int seriesOfQuotesLength = 0;
    int lastCharIsQuote = 0;
    int isEndOfFile = 0;
    while (1) {
        char currChar = (csvParser->fromString_) ? csvParser->csvString_[csvParser->csvStringIter_] : fgetc(csvParser->fileHandler_);
        csvParser->csvStringIter_++;
        int endOfFileIndicator;
        if (csvParser->fromString_) {
            endOfFileIndicator = (currChar == '\\0');
        } else {
            endOfFileIndicator = feof(csvParser->fileHandler_);
        }
        if (endOfFileIndicator) {
            if (currFieldCharIter == 0 && fieldIter == 0) {
                _CsvParser_setErrorMessage(csvParser, "Reached EOF");
                return NULL;
            }
            currChar = '\\n';
            isEndOfFile = 1;
        }
        if (currChar == '\\r') {
            continue;
        }
        if (currFieldCharIter == 0  && ! lastCharIsQuote) {
            if (currChar == '\\"') {
                inside_complex_field = 1;
                lastCharIsQuote = 1;
                continue;
            }
        } else if (currChar == '\\"') {
            seriesOfQuotesLength++;
            inside_complex_field = (seriesOfQuotesLength % 2 == 0);
            if (inside_complex_field) {
                currFieldCharIter--;
            }
        } else {
            seriesOfQuotesLength = 0;
        }
        if (isEndOfFile || ((currChar == csvParser->delimiter_ || currChar == '\\n') && ! inside_complex_field) ){
            currField[lastCharIsQuote ? currFieldCharIter - 1 : currFieldCharIter] = '\\0';
            csvRow->fields_[fieldIter] = (char*)malloc(currFieldCharIter + 1);
            strcpy(csvRow->fields_[fieldIter], currField);
            free(currField);
            csvRow->numOfFields_++;
            if (currChar == '\\n') {
                return csvRow;
            }
            if (csvRow->numOfFields_ != 0 && csvRow->numOfFields_ % acceptedFields == 0) {
                csvRow->fields_ = (char**)realloc(csvRow->fields_, ((numRowRealloc + 2) * acceptedFields) * sizeof(char*));
                numRowRealloc++;
            }
            acceptedCharsInField = 64;
            currField = (char*)malloc(acceptedCharsInField);
            currFieldCharIter = 0;
            fieldIter++;
            inside_complex_field = 0;
        } else {
            currField[currFieldCharIter] = currChar;
            currFieldCharIter++;
            if (currFieldCharIter == acceptedCharsInField - 1) {
                acceptedCharsInField *= 2;
                currField = (char*)realloc(currField, acceptedCharsInField);
            }
        }
        lastCharIsQuote = (currChar == '\\"') ? 1 : 0;
    }
}

int _CsvParser_delimiterIsAccepted(const char *delimiter) {
    char actualDelimiter = *delimiter;
    if (actualDelimiter == '\\n' || actualDelimiter == '\\r' || actualDelimiter == '\\0' ||
            actualDelimiter == '\\"') {
        return 0;
    }
    return 1;
}

void _CsvParser_setErrorMessage(CsvParser *csvParser, const char *errorMessage) {
    if (csvParser->errMsg_ != NULL) {
        free(csvParser->errMsg_);
    }
    int errMsgLen = strlen(errorMessage);
    csvParser->errMsg_ = (char*)malloc(errMsgLen + 1);
    strcpy(csvParser->errMsg_, errorMessage);
}

const char *CsvParser_getErrorMessage(CsvParser *csvParser) {
    return csvParser->errMsg_;
}


// newly added for running the code
char* read_input() {
    int counter = 0;
    char* chars = malloc(sizeof(char) * 1000);
    char c = 0;
    while((c = fgetc(stdin)) != EOF){
        if (counter == 1000) {
            exit(1);
        }
        chars[counter++] = c;
    }
    chars[counter] = '\\0';
    return chars;
}


int main(int argc, char* argv[]) {
    char myString[10000];
    strcpy(myString, argv[1]);
    CsvParser *csvparser = CsvParser_new_from_string(myString, NULL, 1);
    CsvRow *header;
    CsvRow *row;
    int i;

    header = CsvParser_getHeader(csvparser);
    if (header == NULL) {
        printf("%s\\n", CsvParser_getErrorMessage(csvparser));
        return 1;
    }
    char **headerFields = CsvParser_getFields(header);
    for (i = 0 ; i < CsvParser_getNumFields(header) ; i++) {
        printf("TITLE: %s\\n", headerFields[i]);
    }
    while ((row = CsvParser_getRow(csvparser)) ) {
        printf("NEW LINE:\\n");
        char **rowFields = CsvParser_getFields(row);
        for (i = 0 ; i < CsvParser_getNumFields(row) ; i++) {
            printf("FIELD: %s\\n", rowFields[i]);
        }
        CsvParser_destroy_row(row);
    }
    CsvParser_destroy(csvparser);
    return 0;
}

#ifdef __cplusplus
}
#endif
"""

In [ ]:
with open('subjects/csvparser.c', 'w+') as f:
    print(csv_parse, file=f)

### INIParser.c

In [ ]:
ini_parse = """\
/* inih -- simple .INI file parser

inih is released under the New BSD license (see LICENSE.txt). Go to the project
home page for more info:

https://github.com/benhoyt/inih

*/

#ifndef __INI_H__
#define __INI_H__

/* Make this header file easier to include in C++ code */
#ifdef __cplusplus
extern "C" {
#endif

#include <stdio.h>

/* Nonzero if ini_handler callback should accept lineno parameter. */
#ifndef INI_HANDLER_LINENO
#define INI_HANDLER_LINENO 0
#endif

/* Typedef for prototype of handler function. */
#if INI_HANDLER_LINENO
typedef int (*ini_handler)(void* user, const char* section,
                           const char* name, const char* value,
                           int lineno);
#else
typedef int (*ini_handler)(void* user, const char* section,
                           const char* name, const char* value);
#endif

/* Typedef for prototype of fgets-style reader function. */
typedef char* (*ini_reader)(char* str, int num, void* stream);

/* Parse given INI-style file. May have [section]s, name=value pairs
   (whitespace stripped), and comments starting with ';' (semicolon). Section
   is "" if name=value pair parsed before any section heading. name:value
   pairs are also supported as a concession to Python's configparser.

   For each name=value pair parsed, call handler function with given user
   pointer as well as section, name, and value (data only valid for duration
   of handler call). Handler should return nonzero on success, zero on error.

   Returns 0 on success, line number of first error on parse error (doesn't
   stop on first error), -1 on file open error, or -2 on memory allocation
   error (only when INI_USE_STACK is zero).
*/
int ini_parse(const char* filename, ini_handler handler, void* user);

/* Same as ini_parse(), but takes a FILE* instead of filename. This doesn't
   close the file when it's finished -- the caller must do that. */
int ini_parse_file(FILE* file, ini_handler handler, void* user);

/* Same as ini_parse(), but takes an ini_reader function pointer instead of
   filename. Used for implementing custom or string-based I/O (see also
   ini_parse_string). */
int ini_parse_stream(ini_reader reader, void* stream, ini_handler handler,
                     void* user);

/* Same as ini_parse(), but takes a zero-terminated string with the INI data
instead of a file. Useful for parsing INI data from a network socket or
already in memory. */
int ini_parse_string(const char* string, ini_handler handler, void* user);

/* Nonzero to allow multi-line value parsing, in the style of Python's
   configparser. If allowed, ini_parse() will call the handler with the same
   name for each subsequent line parsed. */
#ifndef INI_ALLOW_MULTILINE
#define INI_ALLOW_MULTILINE 1
#endif

/* Nonzero to allow a UTF-8 BOM sequence (0xEF 0xBB 0xBF) at the start of
   the file. See https://github.com/benhoyt/inih/issues/21 */
#ifndef INI_ALLOW_BOM
#define INI_ALLOW_BOM 1
#endif

/* Chars that begin a start-of-line comment. Per Python configparser, allow
   both ; and # comments at the start of a line by default. */
#ifndef INI_START_COMMENT_PREFIXES
#define INI_START_COMMENT_PREFIXES ";#"
#endif

/* Nonzero to allow inline comments (with valid inline comment characters
   specified by INI_INLINE_COMMENT_PREFIXES). Set to 0 to turn off and match
   Python 3.2+ configparser behaviour. */
#ifndef INI_ALLOW_INLINE_COMMENTS
#define INI_ALLOW_INLINE_COMMENTS 1
#endif
#ifndef INI_INLINE_COMMENT_PREFIXES
#define INI_INLINE_COMMENT_PREFIXES ";"
#endif

/* Nonzero to use stack for line buffer, zero to use heap (malloc/free). */
#ifndef INI_USE_STACK
#define INI_USE_STACK 1
#endif

/* Maximum line length for any line in INI file (stack or heap). Note that
   this must be 3 more than the longest line (due to '\\r', '\\n', and '\\0'). */
#ifndef INI_MAX_LINE
#define INI_MAX_LINE 200
#endif

/* Nonzero to allow heap line buffer to grow via realloc(), zero for a
   fixed-size buffer of INI_MAX_LINE bytes. Only applies if INI_USE_STACK is
   zero. */
#ifndef INI_ALLOW_REALLOC
#define INI_ALLOW_REALLOC 0
#endif

/* Initial size in bytes for heap line buffer. Only applies if INI_USE_STACK
   is zero. */
#ifndef INI_INITIAL_ALLOC
#define INI_INITIAL_ALLOC 200
#endif

/* Stop parsing on first error (default is to keep parsing). */
#ifndef INI_STOP_ON_FIRST_ERROR
#define INI_STOP_ON_FIRST_ERROR 1
#endif

#ifdef __cplusplus
}
#endif

#endif /* __INI_H__ */


/* inih -- simple .INI file parser

inih is released under the New BSD license (see LICENSE.txt). Go to the project
home page for more info:

https://github.com/benhoyt/inih

*/

#if defined(_MSC_VER) && !defined(_CRT_SECURE_NO_WARNINGS)
#define _CRT_SECURE_NO_WARNINGS
#endif

// #include <stdio.h>
#include <ctype.h>
#include <string.h>
#include <stdlib.h>

// #include "ini.h"

#if !INI_USE_STACK
#include <stdlib.h>
#endif

#define MAX_SECTION 50
#define MAX_NAME 50

/* Used by ini_parse_string() to keep track of string parsing state. */
typedef struct {
    const char* ptr;
    size_t num_left;
} ini_parse_string_ctx;

/* Strip whitespace chars off end of given string, in place. Return s. */
static char* rstrip(char* s)
{
    char* p = s + strlen(s);
    while (p > s && isspace((unsigned char)(*--p)))
        *p = '\\0';
    return s;
}

/* Return pointer to first non-whitespace char in given string. */
static char* lskip(const char* s)
{
    while (*s && isspace((unsigned char)(*s)))
        s++;
    return (char*)s;
}

/* Return pointer to first char (of chars) or inline comment in given string,
   or pointer to null at end of string if neither found. Inline comment must
   be prefixed by a whitespace character to register as a comment. */
static char* find_chars_or_comment(const char* s, const char* chars)
{
#if INI_ALLOW_INLINE_COMMENTS
    int was_space = 0;
    while (*s && (!chars || !strchr(chars, *s)) &&
           !(was_space && strchr(INI_INLINE_COMMENT_PREFIXES, *s))) {
        was_space = isspace((unsigned char)(*s));
        s++;
    }
#else
    while (*s && (!chars || !strchr(chars, *s))) {
        s++;
    }
#endif
    return (char*)s;
}

/* Version of strncpy that ensures dest (size bytes) is null-terminated. */
static char* strncpy0(char* dest, const char* src, size_t size)
{
    strncpy(dest, src, size - 1);
    dest[size - 1] = '\\0';
    return dest;
}

/* See documentation in header file. */
int ini_parse_stream(ini_reader reader, void* stream, ini_handler handler,
                     void* user)
{
    /* Uses a fair bit of stack (use heap instead if you need to) */
#if INI_USE_STACK
    char line[INI_MAX_LINE];
    int max_line = INI_MAX_LINE;
#else
    char* line;
    int max_line = INI_INITIAL_ALLOC;
#endif
#if INI_ALLOW_REALLOC && !INI_USE_STACK
    char* new_line;
    int offset;
#endif
    char section[MAX_SECTION] = "";
    char prev_name[MAX_NAME] = "";

    char* start;
    char* end;
    char* name;
    char* value;
    int lineno = 0;
    int error = 0;

#if !INI_USE_STACK
    line = (char*)malloc(INI_INITIAL_ALLOC);
    if (!line) {
        return -2;
    }
#endif

#if INI_HANDLER_LINENO
#define HANDLER(u, s, n, v) handler(u, s, n, v, lineno)
#else
#define HANDLER(u, s, n, v) handler(u, s, n, v)
#endif

    /* Scan through stream line by line */
    while (reader(line, max_line, stream) != NULL) {
#if INI_ALLOW_REALLOC && !INI_USE_STACK
        offset = strlen(line);
        while (offset == max_line - 1 && line[offset - 1] != '\\n') {
            max_line *= 2;
            if (max_line > INI_MAX_LINE)
                max_line = INI_MAX_LINE;
            new_line = realloc(line, max_line);
            if (!new_line) {
                free(line);
                return -2;
            }
            line = new_line;
            if (reader(line + offset, max_line - offset, stream) == NULL)
                break;
            if (max_line >= INI_MAX_LINE)
                break;
            offset += strlen(line + offset);
        }
#endif

        lineno++;

        start = line;
#if INI_ALLOW_BOM
        if (lineno == 1 && (unsigned char)start[0] == 0xEF &&
                           (unsigned char)start[1] == 0xBB &&
                           (unsigned char)start[2] == 0xBF) {
            start += 3;
        }
#endif
        start = lskip(rstrip(start));

        if (strchr(INI_START_COMMENT_PREFIXES, *start)) {
            /* Start-of-line comment */
        }
#if INI_ALLOW_MULTILINE
        else if (*prev_name && *start && start > line) {
            /* Non-blank line with leading whitespace, treat as continuation
               of previous name's value (as per Python configparser). */
            if (!HANDLER(user, section, prev_name, start) && !error)
                error = lineno;
        }
#endif
        else if (*start == '[') {
            /* A "[section]" line */
            end = find_chars_or_comment(start + 1, "]");
            if (*end == ']') {
                *end = '\\0';
                strncpy0(section, start + 1, sizeof(section));
                *prev_name = '\\0';
            }
            else if (!error) {
                /* No ']' found on section line */
                error = lineno;
            }
        }
        else if (*start) {
            /* Not a comment, must be a name[=:]value pair */
            end = find_chars_or_comment(start, "=:");
            if (*end == '=' || *end == ':') {
                *end = '\\0';
                name = rstrip(start);
                value = end + 1;
#if INI_ALLOW_INLINE_COMMENTS
                end = find_chars_or_comment(value, NULL);
                if (*end)
                    *end = '\\0';
#endif
                value = lskip(value);
                rstrip(value);

                /* Valid name[=:]value pair found, call handler */
                strncpy0(prev_name, name, sizeof(prev_name));
                if (!HANDLER(user, section, name, value) && !error)
                    error = lineno;
            }
            else if (!error) {
                /* No '=' or ':' found on name[=:]value line */
                error = lineno;
            }
        }

#if INI_STOP_ON_FIRST_ERROR
        if (error)
            break;
#endif
    }

#if !INI_USE_STACK
    free(line);
#endif

    return error;
}

/* See documentation in header file. */
int ini_parse_file(FILE* file, ini_handler handler, void* user)
{
    return ini_parse_stream((ini_reader)fgets, file, handler, user);
}

/* See documentation in header file. */
int ini_parse(const char* filename, ini_handler handler, void* user)
{
    FILE* file;
    int error;

    file = fopen(filename, "r");
    if (!file)
        return -1;
    error = ini_parse_file(file, handler, user);
    fclose(file);
    return error;
}

/* An ini_reader function to read the next line from a string buffer. This
   is the fgets() equivalent used by ini_parse_string(). */
static char* ini_reader_string(char* str, int num, void* stream) {
    ini_parse_string_ctx* ctx = (ini_parse_string_ctx*)stream;
    const char* ctx_ptr = ctx->ptr;
    size_t ctx_num_left = ctx->num_left;
    char* strp = str;
    char c;

    if (ctx_num_left == 0 || num < 2)
        return NULL;

    while (num > 1 && ctx_num_left != 0) {
        c = *ctx_ptr++;
        ctx_num_left--;
        *strp++ = c;
        if (c == '\\n')
            break;
        num--;
    }

    *strp = '\\0';
    ctx->ptr = ctx_ptr;
    ctx->num_left = ctx_num_left;
    return str;
}

/* See documentation in header file. */
int ini_parse_string(const char* string, ini_handler handler, void* user) {
    ini_parse_string_ctx ctx;

    ctx.ptr = string;
    ctx.num_left = strlen(string);
    return ini_parse_stream((ini_reader)ini_reader_string, &ctx, handler,
                            user);
}

// newly added for running the code
typedef struct
{
    int empty;
} configuration;

// took the example from the README file to have sth. to parse
static int handler(void* user, const char* section, const char* name,
                   const char* value)
{
    return 1;
}

int main(int argc, char* argv[]) {
    configuration config;
    return ini_parse_string(argv[1], handler, &config);

}
"""

In [ ]:
with open('subjects/ini.c', 'w+') as f:
    print(ini_parse, file=f)

### Json.c

In [ ]:
vector_h = """\
#ifndef HS_VECTOR_H
#define HS_VECTOR_H

#include <stddef.h>

typedef struct {
    size_t capacity;
    size_t data_size;
    size_t size;
    char* data;
} vector;

void vector_init(vector* v, size_t data_size);

void vector_free(vector* v);

void* vector_get(const vector* v, size_t index);

void* vector_get_checked(const vector* v, size_t index);

void vector_reserve(vector* v, size_t new_capacity);

void vector_push_back(vector* v, void* data);


typedef void(*vector_foreach_t)(void*);

void vector_foreach(const vector* v, vector_foreach_t fp);

typedef int(*vector_foreach_data_t)(void*, void*);

void vector_foreach_data(const vector* v, vector_foreach_data_t fp, void* data);

#ifdef BUILD_TEST
void vector_test_all();
#endif

#endif


#ifndef HS_JSON_H
#define HS_JSON_H

#define BUFFER 10240

/*
enum json_value_type {
	TYPE_NULL,
	TYPE_BOOL,
	TYPE_NUMBER,
	TYPE_OBJECT, // Is a vector with pairwise entries, key, value
	TYPE_ARRAY, // Is a vector, all entries are plain 
	TYPE_STRING,
	TYPE_KEY
};*/

typedef struct {
	int type;
	union {
		int boolean;
		double number;
		char* string;
		char* key;
		vector array;
		vector object;
	} value;
} json_value;

// Parse string into structure of json elements and values
// return 1 if successful.
int json_parse(const char* input, json_value* root);

// Free the structure and all the allocated values
void json_free_value(json_value* val);

// Convert value to string if possible, asserts if not
char* json_value_to_string(json_value* value);

// Convert value to double if possible asserts if not
double json_value_to_double(json_value* value);

// Convert value to bool if possible asserts if not
int json_value_to_bool(json_value* value);

// Convert value to vector if it's an array asserts if not
vector* json_value_to_array(json_value* value);

// Convert value to vector if it's an object, asserts if not
vector* json_value_to_object(json_value* value);

// Fetch the value with given index from root, asserts if root is not array
json_value* json_value_at(const json_value* root, size_t index);

// Fetche the value with the given key from root, asserts if root is not object
json_value * json_value_with_key(const json_value * root, const char * key);

#endif
"""

In [ ]:
with open('subjects/vector.h', 'w+') as f:
    print(vector_h, file=f)

In [ ]:
vector_c = """\
#include "vector.h"

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// Allocate the data structure for the vector
void vector_init(vector* v, size_t data_size) {
	if (v == NULL) return;
	
	v->data = malloc(data_size);
	if (v->data != NULL)
	{
		v->capacity = 1;
        v->data_size = data_size;
        v->size = 0; 
	}
}

// Free the memory of the vector, the pointer to the vector is invalid after this
void vector_free(vector* v)
{
    if (v)
    {
        free(v->data);
		v->data = NULL;
    }
}

// Return the element at index, does not do a range check
void* vector_get(const vector* v, size_t index) {
	return &(v->data[index * v->data_size]);
}

// Return the element at index, return NULL if index is out of range for the vector
void* vector_get_checked(const vector* v, size_t index) {
	return (index < v->size) ? &(v->data[index * v->data_size]) : NULL;
}

// if capacity < new_capacity realloc up to new_capacity
void vector_reserve(vector* v, size_t new_capacity) {
	if (new_capacity <= v->capacity) return;
    void* new_data = realloc(v->data, new_capacity*v->data_size);
    if (new_data) {
        v->capacity = new_capacity;
        v->data = new_data;
    }
    else {
        abort();
    }
}

// Puts an element data[size * data_size], will reserve more space if size == capacity
void vector_push_back(vector* v, void* data) {
    if (v->size >= v->capacity) {
		size_t new_capacity = (v->capacity > 0) ? (size_t)(v->capacity * 2) : 1;
		vector_reserve(v, new_capacity);
    }
    memcpy(vector_get(v,v->size), data, v->data_size);
    ++v->size;
}

void vector_foreach_data(const vector* v, vector_foreach_data_t fp, void* data)
{
	if (v == NULL) return;
	char* item = v->data;
	assert(item != NULL);
	for (size_t i = 0; i < v->size; i++) {
		if (! fp(item, (void *)data)) break;
		item += v->data_size;
	}
}

void vector_foreach(const vector* v, vector_foreach_t fp)
{
	if (v == NULL) return;
	char* item = v->data;
	assert(item != NULL);
	for (size_t i = 0; i < v->size; i++) {
		fp(item);
		item += v->data_size;
	}
}

#ifdef BUILD_TEST

void vector_test_alloc_free(void)
{
	printf("vector_test_alloc_free: ");
	vector v; 
	vector_init(&v, sizeof(int));
    assert(v.capacity == 1);
    assert(v.size == 0);
    assert(v.data_size == sizeof(int));
    assert(v.data != NULL);
	printf("OK\\n");
    vector_free(&v);
}

void vector_test_insert_read_int(void)
{
	printf("vector_test_insert_read_int: ");
	int val1 = 0xabcdabcd;
	int val2 = 0xeffeeffe;
	vector v; 
	vector_init(&v, sizeof(int));
	vector_push_back(&v,&val1);
	assert(v.size == 1);
	assert(v.capacity == 1);
	int* p = vector_get(&v, 0);
	assert(*p == val1);
	vector_push_back(&v, &val2);
	p = vector_get(&v, 0);
	assert(*p == val1);
	assert(*(p + 1) == val2);

	printf("OK\\n");
	vector_free(&v);
}

void vector_test_insert_read_struct(void)
{
	struct data {
		double d;
		int i;
	};

	printf("vector_test_insert_read_struct: ");
	vector v; 
	vector_init(&v, sizeof(struct data));
	struct data d1 = { 0.05, 123 };
	struct data d2 = { -1.9999e10, -9000 };
	vector_push_back(&v, &d1);
	vector_push_back(&v, &d2);
	struct data* p = vector_get(&v, 0);
	assert((*p).d == d1.d); // Bitcopy should be exactly equal
	assert((*p).i == d1.i);
	p = vector_get(&v, 1);
	assert((*p).i == d2.i);
	assert((*p).d == d2.d);

	printf("OK\\n");
	vector_free(&v);
}

void vector_test_safe_get(void)
{
	printf("vector_test_safe_get:  ");
	vector v; 
	vector_init(&v, sizeof(int));
	int val = 0xff;
	vector_push_back(&v, &val);
	vector_push_back(&v, &val);

	assert(NULL == vector_get_checked(&v, -1));
	assert(NULL == vector_get_checked(&v, 2));
	assert(val == *(int*)(vector_get_checked(&v, 1)));

	printf("OK\\n");
	vector_free(&v);
}

void vector_test_reserve(void)
{
	printf("vector_test_reserve:  ");
	vector v; 
	vector_init(&v, sizeof(int));
	assert(v.capacity == 1);
	vector_reserve(&v, 10);
	assert(v.capacity == 10);

	// if we didn't assign the correct space VS will shout about overwriting memory in DEBUG
	int* p = (int*)v.data;
	for (int i = 0; i < 10; ++i) {
		*p = i;
		++p;
	}

	printf("OK\\n");
	vector_free(&v);
}

void foreach_increment_nodata(void* item)
{
	assert(item != NULL);
	int* val = item;
	*val = *val + 1;
}


void vector_test_foreach_nodata(void)
{
	vector v;
	printf("vector_test_foreach_nodata: ");

	vector_init(&v, sizeof(int));
	int val = 0;

	for (size_t i = 0; i < 5; ++i)
	{
		vector_push_back(&v, &val);
		++val;
	}

	vector_foreach(&v, foreach_increment_nodata);

	for (size_t i = 0; i < 5; ++i)
	{
		int* d = vector_get(&v, i);
		assert(*d == i + 1);
	}

	printf("OK\\n");
	vector_free(&v);
}


int foreach_increment_data_null(void* item, void* data)
{
	assert(item != NULL);
	assert(data == NULL);
	int* val = item;
	*val = *val + 1;
	return 1;
}


void vector_test_foreach_data_1(void)
{
	vector v;	
	printf("vector_test_foreach_data_1: ");

	vector_init(&v, sizeof(int));
	int val = 0;

	for (size_t i = 0; i < 5; ++i) {
		vector_push_back(&v, &val);
		++val;
	}

	vector_foreach_data(&v, foreach_increment_data_null, NULL);

	for (size_t i = 0; i < 5; ++i) {
		int* d = vector_get(&v, i);
		assert(*d == i+1);
	}

	printf("OK\\n");
	vector_free(&v);
}

struct foreach_data {
	int i;
};

int foreach_increment_data(void* item, void* data)
{
	assert(item != NULL);
	assert(data != NULL);
	int *i = item;
	((struct foreach_data*)data)->i += *i;
	return 1;
}

void vector_test_foreach_data_2(void)
{
	vector v;
	printf("vector_test_foreach_data_2: ");

	vector_init(&v, sizeof(int));
	int val = 4;
	int sum = 0;
	for (size_t i = 0; i < 5; ++i) {
		vector_push_back(&v, &val);
		sum += val;
		++val;
	}

	struct foreach_data data = {.i=0};
	vector_foreach_data(&v, foreach_increment_data, &data);
	assert(data.i == sum);

	printf("OK\\n");
	vector_free(&v);
}

void vector_test_all(void)
{
    vector_test_alloc_free();
	vector_test_insert_read_int();
	vector_test_insert_read_struct();
	vector_test_safe_get();
	vector_test_reserve();
	vector_test_foreach_nodata();
	vector_test_foreach_data_1();
	vector_test_foreach_data_2();
}

#endif // UNIT_TEST
"""

In [ ]:
with open('subjects/vector.c', 'w+') as f:
    print(vector_c, file=f)

In [ ]:
json_c = """\
#include "vector.h"
/* From
 * https://raw.githubusercontent.com/HarryDC/JsonParser/53197e0b84a7c8d3d57b840e40ea13efdd01d57d/src/json.c*/
/*I have no idea why CLANG has a problem with enumed case labels
TYPE_NULL = 0;
TYPE_BOOL = 1;
TYPE_NUMBER = 2;
TYPE_OBJECT = 3;
TYPE_ARRAY = 4;
TYPE_STRING = 5;
TYPE_KEY = 6;*/

#include <assert.h>
#include <ctype.h>
#include <stddef.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

static int json_parse_value(const char **cursor, json_value *parent);

int isspace_(char c) {
  return isspace(c);
}

int iscntrl_(char c) {
  return iscntrl(c);
}

static void skip_whitespace(const char **cursor) {
  if (**cursor == '\\0')
    return;
  while (iscntrl_(**cursor) || isspace_(**cursor))
    ++(*cursor);
}

static int has_char(const char **cursor, char character) {
  skip_whitespace(cursor);
  int success = **cursor == character;
  if (success)
    ++(*cursor);
  return success;
}

static int json_parse_object(const char **cursor, json_value *parent) {
  json_value result;
  result.type = 3; /*TYPE_OBJECT*/;
  /*vector_init(&result.value.object, sizeof(json_value));*/

  int success = 1;
  while (success && !has_char(cursor, '}')) {
    json_value key;
    key.type = 0; /*TYPE_NULL*/
    json_value value;
    value.type = 0; /*TYPE_NULL*/
    success = json_parse_value(cursor, &key);
    success = success && has_char(cursor, ':');
    success = success && json_parse_value(cursor, &value);

    if (success) {
      /*vector_push_back(&result.value.object, &key);
      vector_push_back(&result.value.object, &value);*/
    } else {
      json_free_value(&key);
      break;
    }
    skip_whitespace(cursor);
    if (has_char(cursor, '}'))
      break;
    else if (has_char(cursor, ','))
      continue;
    else
      success = 0;
  }

  if (success) {
    *parent = result;
  } else {
    json_free_value(&result);
  }

  return success;
}

static int json_parse_array(const char **cursor, json_value *parent) {
  int success = 1;
  if (**cursor == ']') {
    ++(*cursor);
    return success;
  }
  while (success) {
    json_value new_value;
    new_value.type = 0; /*TYPE_NULL*/
    success = json_parse_value(cursor, &new_value);
    if (!success)
      break;
    skip_whitespace(cursor);
    /*vector_push_back(&parent->value.array, &new_value);*/
    skip_whitespace(cursor);
    if (has_char(cursor, ']'))
      break;
    else if (has_char(cursor, ','))
      continue;
    else
      success = 0;
  }
  return success;
}

void json_free_value(json_value *val) {
  if (!val)
    return;

  switch (val->type) {
  case 5 /*TYPE_STRING*/: {
    free(val->value.string);
    val->value.string = 0;
    break;
  }
  case 4 /*TYPE_ARRAY*/: {
    /*vector_foreach(&(val->value.array), (void (*)(void *))json_free_value);
    vector_free(&(val->value.array));*/
    break;
  }
  case 3 /*TYPE_OBJECT*/: {
    /*vector_foreach(&(val->value.array), (void (*)(void *))json_free_value);
    vector_free(&(val->value.array));*/
    break;
  }
  }

  val->type = 0 /*TYPE_NULL*/;
}

int json_is_literal(const char **cursor, const char *literal) {
  size_t cnt = strlen(literal);
  if (strncmp(*cursor, literal, cnt) == 0) {
    *cursor += cnt;
    return 1;
  }
  return 0;
}

static int json_parse_value(const char **cursor, json_value *parent) {
  /* Eat whitespace */
  int success = 0;
  skip_whitespace(cursor);
  switch (**cursor) {
  case '\\0': {
    /* If parse_value is called with the cursor at the end of the string that's a failure*/
    success = 0;
    break;
  }
  case '"': {
    ++*cursor;
    const char *start = *cursor;
    char *end = (char*) strchr(*cursor, '"');
    if (end) {
      size_t len = end - start;
      char *new_string = (char*) malloc((len + 1) * sizeof(char));
      memcpy(new_string, start, len);
      new_string[len] = '\\0';

      if (len != strlen(new_string)) {
        exit(100);
      }

      parent->type = 5 /*TYPE_STRING*/;
      parent->value.string = new_string;

      *cursor = end + 1;
      success = 1;
    }
    break;
  }
  case '{': {
    ++(*cursor);
    skip_whitespace(cursor);
    success = json_parse_object(cursor, parent);
    break;
  }
  case '[': {
    parent->type = 4 /*TYPE_ARRAY*/;
    /*vector_init(&parent->value.array, sizeof(json_value));*/
    ++(*cursor);
    skip_whitespace(cursor);
    success = json_parse_array(cursor, parent);
    if (!success) {
      /*vector_free(&parent->value.array);*/
    }
    break;
  }
  case 't': {
    success = json_is_literal(cursor, "true");
    if (success) {
      parent->type = 1; /* TYPE_BOOL;*/
      parent->value.boolean = 1;
    }
    break;
  }
  case 'f': {
    success = json_is_literal(cursor, "false");
    if (success) {
      parent->type = 1; /* TYPE_BOOL;*/
      parent->value.boolean = 0;
    }
    break;
  }
  case 'n': {
    success = json_is_literal(cursor, "null");
    break;
  }
  default: {
    char *end;
    double number = strtod(*cursor, &end);
    if (*cursor != end) {
      parent->type = 2 /*TYPE_NUMBER*/;
      parent->value.number = number;
      *cursor = end;
      success = 1;
    }
  }
  }

  return success;
}

int json_parse(const char *input, json_value *result) {
  const char** cursor = &input;
  int val = json_parse_value(cursor, result);
  if (strlen(*cursor) != 0){
    return 0;
  }
  return val;
}

int main(int argc, char *argv[]) {
  char my_string[10240];
  json_value result;
  result.type = 0;/*TYPE_NULL*/
  int ret = -1;
  
  strcpy(my_string, argv[1]);
  ret = json_parse(my_string, &result);
  json_free_value(&result);
  
  return 0;
}
"""

In [ ]:
with open('subjects/json.c', 'w+') as f:
    print(json_c, file=f)

### Subject Registry

We store all our subject programs under `program_src`.

In [ ]:
program_src = {
    'calculator.c': 'calc_parse',
    'cgidecode.c': 'cgi_decode',
    'url_parse.c': 'url_parse',
    'ini.c': 'ini_parse',
    'csvparser.c': 'csv_parse'
}

## Deriving Input Grammar from non-Stripped Binaries 

Non-stripped binaries are binaries that have debugging information built into it, which basically means if an executable is compiled with `gcc`'s `-g` flag, it contains debugging information. In order to derive an input from a non-stripped binary, we build a debugger which leverage the debugging information presence in the binary in order to produce an input grammar.

In order to be able  recover an input grammar. We need to hook into the program runtime observe debugging information such as the arguments to a function, local variables and the context of execution by inspecting the currently observed `frame`.

To do this, we make use of `GDB`, the GNU Project debugger which allow us to trace through a program and also gives us access to the same contextual information as we have seen previously in the example implemented in python.`GDB` provides a python API which can be use in debugging binary programs and also accessing program information at runtime. 

### Debugger

Before we can start debugging programs, we use `GDB` python API to implement a debugger that would suit our aim by  creating an interface that provides functions which a standard debugger would have.

In [ ]:
class Debugger(object):
    def run(self):
        raise NotImplementedError()

    def step(self):
        raise NotImplementedError()

    def break_at(self, line):
        raise NotImplementedError()

    def start_program(self, inp, binary):
        raise NotImplementedError()

    def event_loop(self):
        raise NotImplementedError()

#### GDBDebugger
We provides a concrete implementation of the above interface.

In [ ]:
class GDBDebugger(Debugger):
    def __init__(self, gdb, binary, inp, **kwargs):
        self.options(kwargs)
        self.gdb, self.binary, self.inp = gdb, binary, inp
        self.frames = []
        self._set_printer()
        self._set_logger()
        self._skip_std_files()
        self.tracer = GDBTracer(self.inp, files=self.files)

In [ ]:
class GDBDebugger(GDBDebugger):
    def run(self):
        self.gdb.execute('run')

In [ ]:
class GDBDebugger(GDBDebugger):
    def step(self):
        self.gdb.execute('step')

In [ ]:
class GDBDebugger(GDBDebugger):
    def break_at(self, line):
        self.gdb.execute("break '%s'" % line)
        self.run()

In [ ]:
class GDBDebugger(GDBDebugger):
    def start_program(self, inp, binary):
        self.gdb.execute("set args '%s'" % inp)
        self.gdb.execute("file %s" % binary)

The `_set_logger` function is used to configure logging.

In [ ]:
class GDBDebugger(GDBDebugger):
    def _set_logger(self):
        self.gdb.execute('set logging overwrite on')
        self.gdb.execute('set logging redirect on')
        self.gdb.execute('set logging on')

When stepping through binary programs, we need to make sure we avoid stepping into files which are not of interest to us such as the *standard libraries files*. One of the ways to do this is to tell `GDB` to skip all files which are of a particular format, an example is the *.S files.

In [ ]:
class GDBDebugger(GDBDebugger):
    def _set_printer(self):
        if not self.printer:
            self.gdb.execute('set print address off')

Furthermore, in order to efficiently avoid stepping into files we are not interested, we define a variable `file` which holds an array of file names which we are interested in tracing.

In [ ]:
class GDBDebugger(GDBDebugger):
    def _skip_std_files(self):
        self.gdb.execute('skip -gfi *.S')

Furthermore, in order to efficiently avoid stepping into files we are not interested, we define a variable `file` which holds an array of file names which we are interested in tracing.

In [ ]:
class GDBDebugger(GDBDebugger):
    def options(self, kwargs):
        self.files = kwargs.get('files', [])
        self.methods = kwargs.get('methods', [])
        self.printer = kwargs.get('printer', True)

At each step in our program we need to always check if we are within the context which we are most interested in. To do this, we provide the function `in_context` which take the selected frame at each *step* in our program an then to a check if we are still within the context that we are interested.

In [ ]:
class GDBDebugger(GDBDebugger):
    def in_context(self, frame):
        file_name = frame.find_sal().symtab.fullname()
        return any(file_name.endswith(f) for f in self.files)

The function `get_event` keeps track of how frame are being created when stepping through our program. The idea behind this is that whenever a frame is newly created, it is added to the frame list and that shows that a function call has occurred within our program then we assign the event as a `call`. Also, whenever a frame is the last frame being added to our frame list that also implies that we are still within that particular frame  and then we assign the event as `line`. Lastly, if none of the above has occurred then it shows that particular frame has exited and then we assign our event as `return`.

In [ ]:
class GDBDebugger(GDBDebugger):
    def get_event(self, frame):
        fname = frame.name()
        if fname not in self.frames:
            self.frames.append(fname)
            return 'call'
        elif fname == self.frames[-1]:
            return 'line'
        else:
            self.frames.pop()
            return 'return'

The `event_loop` starts our program and the auto-step through our program while it runs. Once we start our program we get the selected frame and then assign it to the variable called `frame`. The `frame` variable in gdb automatically becomes `False` when the program exits even though we never explicitly assign to it. Also, at each step in our program we check if there is a new frame and if the frame is within our scope of interest. If not, we instruct `GDB` to finish execution from the uninterested scope and then returns back to it's caller.

In [ ]:
class GDBDebugger(GDBDebugger):
    def event_loop(self):
        self.start_program(self.inp, self.binary)
        self.break_at('main')
        frame = self.gdb.selected_frame()
        try:
            while frame.is_valid():
                if self.gdb.selected_frame() != frame:
                    self.step()
                    current_frame = self.gdb.selected_frame()
                    if not self.in_context(current_frame):
                        # simply finish the current function execution.
                        self.gdb.execute('finish')
                        continue
                    event = self.get_event(current_frame)
                    self.tracer.traceit(current_frame, event, None)
                else:
                    self.step()
                    if not self.in_context(self.gdb.selected_frame()):
                        self.gdb.execute('finish')
        except gdb.error:
            return

### VarExtractor

Next, We also define a class called `VarExtractor` which provides various logic that can be used to extract and process variables which are contained a frame.

In [ ]:
class VarExtractor:
    def __init__(self, frame):
        self.frame = frame

#### Extract integer values
The function `extract_int_val` takes a symbol which type is an integer as argument and then looks up such symbol in the frame and then returns the value of such symbols. This form of extraction works for an integer which does not have a pointer type.

In [ ]:
class VarExtractor(VarExtractor):
    def extract_int_val(self, symbol):
        return '{}'.format(symbol.value(self.frame))

#### Dereference pointer types

`dereference_pointer_type` is solely used for symbols which are of the type `pointer`. This function basically dereference a pointer type and then returns the actual type which it points to.

In [ ]:
class VarExtractor(VarExtractor):
    def dereference_pointer_type(self, symbol):
        return symbol.value(self.frame).dereference()

`extract_struct_val` take a symbol which is of type `struct` and then returns the value as a key, value pair.

In [ ]:
class VarExtractor(VarExtractor):
    def extract_struct_val(self, struct):
        return {
            f.name: str(struct[f]).strip('"')
            for f in struct.type.fields()
        }

#### Extract pointer values

The function `extract_pointer_val` which firstly dereference a pointer type and then returns the true type of the object it points to. Next, we check the type of the object being pointed to and then we extract the value and then return.

In [ ]:
class VarExtractor(VarExtractor):
    def extract_pointer_val(self, symbol):
        true_value = self.dereference_pointer_type(symbol)

        if true_value.type.code == gdb.TYPE_CODE_INT:
            return '{}'.format(true_value.address).strip('"')

        elif true_value.type.code == gdb.TYPE_CODE_STRUCT:
            return self.extract_struct_val(true_value)

### GDBContext

We've seen previously that the `Context` class provides easy access to the information such as the current module, and parameter names. We can also obtain same information using `GDB`  to access the frame as seen below.

We call our new context class `GDBContext`.

In [ ]:
from GrammarMiner import Context

In [ ]:
class GDBContext(Context):
    def __init__(self, frame):
        self.method = frame.name()
        self.parameter_names = self.get_arg_names(frame)
        self.line_no = frame.find_sal().line
        self.file_name = frame.find_sal().symtab.fullname()

The `get_arg_names` is a custom function which takes a `frame` as input, extract the name of the arguments and return a list of argument names. `GDB` represents variables, constants, arguments as symbols in a block. In a more descriptive sense, a block is just a scope in the source code. Also, `gdb.Block` is iterable just as we can see in the `get_arg_names` function.

In [ ]:
class GDBContext(GDBContext):
    def get_arg_names(self, frame):
        return [symbol.name for symbol in frame.block() if symbol.is_argument]

We also extend the `extract_vars` which is a convenience method that acts on the frame within the `GDBContext` class. In this case we iterate through all `symbols` in the current `block`. If the symbol is a variable or an argument, we check what type they carry and then we extract their corresponding values based on their type and then we add them to  dictionary `vals` as defined.

In [ ]:
class GDBContext(GDBContext):
    def extract_vars(self, frame):
        vals = {}
        extractor = VarExtractor(frame)

        symbols = [
            sym for sym in frame.block() if sym.is_variable or sym.is_argument
        ]
        for symbol in symbols:
            if symbol.type.code == gdb.TYPE_CODE_INT:
                vals[symbol.name] = extractor.extract_int_val(symbol)

            elif symbol.type.code == gdb.TYPE_CODE_PTR:
                vals[symbol.name] = extractor.extract_pointer_val(symbol)

        return {k1: v1 for k, v in vals.items() for k1, v1 in flatten(k, v)}

###  GDBTracer

Previously, we have seen how `Tracer` class was used to trace through a python program to obtain the trace information. In our case, we define a new class `GDBTracer` that inherits the base implementation of our `Tracer` class, the only exception we have is to use the `GDBContext` class which we already defined above. To do this, we override the function `create_context` and then return an instance of `GDBContext`.

In [ ]:
from GrammarMiner import Tracer

**IMPORTANT**: verify that your current version of fuzzingbook you use uses `create_context()` in the `traceit()` method.

In [ ]:
class GDBTracer(Tracer):
    def create_context(self, frame):
        return GDBContext(frame)

### Recovering Grammar

#### Putting all together

Due to the limitation we have that `GDB` API is not a library, we cannot execute our implementation directly within the notebook cell. To address this limitation, we use the fuzzingbook `extract_class_definition` function to extract each of the classes we have implemented so far and then we write them to a `.py` file which would then be execute under `GDB`.

In [ ]:
import sys
import inspect

In [ ]:
tracer_head = """\
import sys
sys.path.extend([%s])
sys.path.append('.')
import matplotlib.pyplot
matplotlib.pyplot._IP_REGISTERED = True # Hack
import fuzzingbook_utils
from GrammarMiner import GrammarMiner, Context, Tracer, Coverage, ScopedGrammarMiner, readable, flatten
import jsonpickle
import os
import gdb
""" % (', '.join("'%s'" % str(i) for i in sys.path if i))
debugger_src = fuzzingbook_utils.extract_class_definition(Debugger)
context_src = fuzzingbook_utils.extract_class_definition(GDBContext)
gdbtracer_src = fuzzingbook_utils.extract_class_definition(GDBTracer)
varextractor_src = fuzzingbook_utils.extract_class_definition(VarExtractor)
gdbdebugger_src = fuzzingbook_utils.extract_class_definition(GDBDebugger)
tracer_tail="""
file_name = 'gdbtrace'
def recover_trace(f, inp, **kwargs):
    d = GDBDebugger(gdb, f, inp, **kwargs)
    d.event_loop()
    with open(file_name, 'w+') as f:
        print(jsonpickle.encode(d.tracer.trace), file=f)
binary = 'a.out'
recover_trace(binary, arg0, files=files.split(' '))
"""
tracer_src = '\n'.join([tracer_head, debugger_src, context_src, gdbtracer_src, varextractor_src, gdbdebugger_src, tracer_tail])

In [ ]:
#with open('build/debugger.py', 'w+') as f:
    #print(tracer_src, file=f)

#### The method recover grammar
`recover_grammar` function is similar to the one seen previously. The exception it has is that we execute the `debugger.py` file under `GDB` and then we read the trace before updating the scope grammar miner instance.

In [ ]:
from GrammarMiner import ScopedGrammarMiner, readable
import jsonpickle

In [ ]:
def recover_grammar(inps, src):
    traces = []
    miner = ScopedGrammarMiner()

    for inp in inps:
        arg = '\'py arg0="%s"\'' % inp
        argfiles = '\'py files="%s"\'' % src
        print(arg)
        !gdb --batch-silent -ex {arg} -ex {argfiles} -x debugger.py
        with open('gdbtrace', 'rb') as f:
            traces.append((inp, jsonpickle.decode(f.read())))
    
    for inp, trace in traces:
        miner.update_grammar(inp, trace)
    return (readable(miner.clean_grammar()))

## Deriving Input Grammar from Stripped Binaries

Stripped Binaries are binaries that does not include debugging symbols during program execution. This can be produced by including `-s` flag when compiling the program with `gcc`.
When considering stripped binaries, we would understand that our above implementation cannot directly be used to obtain an input grammar from such binaries since they do not contain debugging information. 

One thing to consider is that stripped binaries only contains sets of `assembly instructions` which are usually executed sequentially at runtime. With this idea, we can observe these sets of instrutions and then infer some useful program informations which would be useful to produce meaningful input grammar.

### Assembly Instructions

An assembly instruction is typically made up of a line address, an instruction type which specifies what type of action of operation to be taken and also a source and a destination registers

Below is an example of an instruction that moves a value from one register to another;

`0x555555554892:      mov    %rsp,%rbp` where `0x555555554892` is the current line address, `mov` is the type of instruction that should be carried out and lastly `%rsp`,`%rbp` which represents the source and destination registers respectively.

#### Assembly Instruction as an Object

With the information we can obtain from each assembly instruction, we represents this as an object called `Instruction`.

In [ ]:
global_declarations = '''
CALL = 'callq'
RETURN = 'retq'
LINE = 'line'
ARG_REGISTERS = ['rdi', 'rsi', 'rdx', 'rcx', 'r8', 'r9']
REGISTERS = ARG_REGISTERS + ['rax', 'eax', 'edi' 'esi', 'edx', 'ecx']
UNWANTED = [
    'leaveq', 'retq', 'nop', 'je', 'jne', 'jmp', 'jle', 'jmpq', 'jae', 'jbe',
    'cltq', 'ja', 'jb', 'js'
]
ADDR_COUNTER_ = 0
F_ADDR_COUNTER_ = 0
L_ADDR_COUNTER = 0
CALL_STORE = []
LEN_OF_ADDR = 14
'''

In [ ]:
class Instruction:
    def __init__(self, instr):
        self.symbol_name = None
        self.pointed_address = None
        self.dest_reg = None
        self.instr_type = None
        self._parse(instr)

The function `_parse()` takes an instruction as string and then extract information such as the current line address, function address, registers as well as instruction type.

In [ ]:
class Instruction(Instruction):
    def _parse(self, instr):
        instr_list = instr.split()
        instr_list.pop(0)

        self.current_address = instr_list[0]
        if "<" in instr_list[1]:
            instr_list.pop(1)
        self.instr_type = instr_list[1]

        if self.instr_type == CALL:
            self.pointed_address = self.get_pointed_value(instr_list[2])
            if len(instr_list) > 3:
                self.symbol_name = instr_list[-1]

        elif self.instr_type in UNWANTED:
            return
        else:
            if len(instr_list) > 2:
                d = instr_list[2]
            else:
                return

            if d[-1] != ')':
                if d[-3:] in REGISTERS:
                    self.dest_reg = d[-3:]
                else:
                    r = instr_list[-1][-3:]
                    self.dest_reg = r if r in REGISTERS else None
            elif re.match(r',\d\)', d[-3:]):
                d = d.split(',')
                if d[0][1:] in REGISTERS:
                    self.dest_reg = d[0][1:]
            else:
                for r in REGISTERS:
                    if r in d:
                        self.dest_reg = r

In [ ]:
class Instruction(Instruction):
    def get_pointed_value(self, val):
        val = val.strip('%*')
        if val in REGISTERS:
            ptr_addr = gdb.execute('x/s $%s' % (val),
                                   to_string=True).split(':')
            return ptr_addr[0]
        return val

### BinaryVarExtractor

The class `BinaryVarExtractor` is a variant of the `VarExtractor` defined above which extracts variable values from X86 register as well as memory address that points to values.

In [ ]:
class BinaryVarExtractor:
    def __init__(self, inp):
        self.inp = inp

In [ ]:
class BinaryVarExtractor(BinaryVarExtractor):
    def read_reg_as_string(self, register):
        str_result = gdb.execute('x/s $%s' % register, to_string=True)
        for index, char in enumerate(str_result):
            if str_result[index] == ':':
                addr = str_result[:index]
                break
        return addr

In [ ]:
class BinaryVarExtractor(BinaryVarExtractor):
    def get_address(self, addr):
        s = gdb.execute('x/a %s' % addr, to_string=True)
        for index, char in enumerate(s):
            if s[index] == ':':
                addr = s[index + 1:]
                break
        return addr

In [ ]:
class BinaryVarExtractor(BinaryVarExtractor):
    def read_address(self, addr):
        if len(addr) == ADDR_LENGTH:
            str_result = gdb.execute('x/s %s' % addr, to_string=True)
            key, value = self.split_string_value(str_result)

            if value not in self.inp:
                addr_result = self.get_address(key)
                s1 = gdb.execute('x/s %s' % addr_result, to_string=True)
                k1, v1 = self.split_string_value(s1)
                return v1
            return value
        return ''

In [ ]:
class BinaryVarExtractor(BinaryVarExtractor):
    def split_string_value(self, strval):
        for count, char in enumerate(strval):
            if strval[count] == ':':
                key = strval[:count]
                value = strval[count + 3:-2]
                break
        return key, value.replace('\\', '')

In [ ]:
class BinaryVarExtractor(BinaryVarExtractor):
    def read_register(self, register):
        addr = self.read_reg_as_string(register)
        return self.read_address(addr)

### Frame

In stripped binaries, we do not have access to stack frame unlike non-stripped binaries.
As a solution, we build a frame class that represents information we would normally have in a stack frame.

The idea of this frame is that whenever we observe a `callq` instruction which represents a `call` event, we instantiate a new frame. All other instruction types are treated as `line` event with the exception of `retq` instruction which represents `return` event.

Furthermore, when there is an `line` event we do not need to create a new frame, instead we update the current frame object such as the local variable, current line address.

In [ ]:
helper_methods = """
ADDR_STORE = {}
F_ADDR_STORE = {}
L_ADDR_STORE = {}
FN_DICT = {}
ADDR_LENGTH = 14

def get_addr(addr, t):
    global ADDR_COUNTER_
    if t == 'arg':
        ADDR_COUNTER_ += 1
        return ADDR_COUNTER_
    if addr in ADDR_STORE: return ADDR_STORE[addr]
    ADDR_COUNTER_ += 1
    ADDR_STORE[addr] = ADDR_COUNTER_
    return ADDR_STORE[addr]
    
def get_fn_addr(addr):
    global F_ADDR_COUNTER_
    if addr in F_ADDR_STORE: return F_ADDR_STORE[addr]
    F_ADDR_COUNTER_ += 1
    F_ADDR_STORE[addr] = F_ADDR_COUNTER_
    return F_ADDR_STORE[addr]
    
def get_fn(fnaddr):
    fn_dict = {}
    with open(f'function_names', 'rb') as f:
        fn_dict = jsonpickle.decode(f.read())
    return fn_dict[fnaddr] if fnaddr in fn_dict.keys() else fnaddr
    
def get_var(varaddr, t):
    aid = get_addr(varaddr, t)
    return "var_%d" % aid
    
def get_lno(addr):
    global L_ADDR_COUNTER
    if addr in L_ADDR_STORE: return 'a_%d' % (L_ADDR_STORE[addr])
    L_ADDR_COUNTER += 1
    L_ADDR_STORE[addr] = L_ADDR_COUNTER
    s = 'a_%d' % (L_ADDR_STORE[addr])
    return s
"""

In [ ]:
class Frame:
    def __init__(self, instr, inp):
        self.inp = inp
        self.arguments = []
        self.locals_vars = {}
        self.line_no = None
        self.file_name = None
        self.function = None
        self.bextr = BinaryVarExtractor(inp)
        self._parse(instr)
        CALL_STORE.append(self)

The function `_read_register()` looks up the content of the register and then return a variable name along with the content of the register as the value.

In [ ]:
class Frame(Frame):
    def _read_register(self, register, l_no, tag):
        str_val = self.bextr.read_register(register)
        var = get_var(l_no, tag)
        return (var, str_val)

The function `_read_arg_register()` is used to get the content of the first 6 registers which by convention is used to store arguments of a function call.

In [ ]:
class Frame(Frame):
    def _read_arg_register(self):
        unique_args = []
        vals = [
            self._read_register(reg, self.line_no, 'arg')
            for reg in ARG_REGISTERS
        ]

        for k, v in vals:
            if not unique_args:
                unique_args.append((k, v))
            else:
                lst = [v for k, v in unique_args]
                if v in lst: continue
                else: unique_args.append((k, v))
        args = {k: v for k, v in unique_args if v in self.inp}
        return args

The function `_parse()` is used to set the information contained in an instruction which is been passed to the frame on instantiation.

In [ ]:
class Frame(Frame):
    def _parse(self, instr):
        self.function = get_fn(instr.pointed_address)
        self.line_no = get_lno(instr.current_address.strip(':'))
        self.arguments = self._read_arg_register()
        self.file_name = 'a.out'
        self.locals_vars.update(self.arguments)
        self.event = 'call'

The function `_get_event()` sets each event based on the type of instruction being executed.

In [ ]:
class Frame(Frame):
    def _get_event(self, instr_type):
        if instr_type == CALL:
            return 'call'
        elif instr_type == RETURN:
            return 'return'
        else:
            return 'line'

The function `update()`update the current frame's local variables, and line address as instructions are being stepped through. Also, it keeps track of all `call` sequence as well as remove the correponding frame whenever there is a `return` event.

In [ ]:
class Frame(Frame):
    def update(self, instr):
        if CALL_STORE and self.function != CALL_STORE[-1].function:
            self.function = CALL_STORE[-1].function
            self.arguments = CALL_STORE[-1].arguments
            self.locals_vars = CALL_STORE[-1].locals_vars

        self.line_no = get_lno(instr.current_address.strip(':'))
        self.event = self._get_event(instr.instr_type)
        if self.event == 'return' and CALL_STORE:
            CALL_STORE.pop()

        if instr.dest_reg is not None:
            var, val = self._read_register(instr.dest_reg,
                                           instr.current_address, 'var')
            self.locals_vars[var] = val

### Context

#### Binary Context

Using the frame object we created, we use this to update our context class just like we did above in the case of the non-stripped binaries implementation.

In [ ]:
class BinaryContext(GDBContext):
    def __init__(self, frame):
        self.method = frame.function
        self.parameter_names = [k for k, v in frame.arguments.items()]
        self.line_no = frame.line_no
        self.file_name = frame.file_name

In [ ]:
class BinaryContext(BinaryContext):
    def extract_vars(self, frame):
        return {k1: v1 for k, v in frame.locals_vars.items() for k1, v1 in flatten(k, v)}

In [ ]:
class BinaryContext(BinaryContext):
    def __repr__(self):
        return "%s:%s:%s(%s)" % self._t()

### Binary Tracer

We update our tracer which basically use our new context class.

In [ ]:
class BinaryTracer(GDBTracer):
    def create_context(self, frame):
        return BinaryContext(frame)

### Debugger

#### Binary Debugger

The binary debugger makes use of `gdb` commands which can be used to step through x86 assembly instructons. 

In [ ]:
class BinaryDebugger(GDBDebugger):
    def __init__(self, gdb, binary, inp, **kwargs):
        super().__init__(gdb, binary, inp, **kwargs)
        self.tracer = BinaryTracer(self.inp, files=self.files)

In [ ]:
class BinaryDebugger(BinaryDebugger):
    def break_at(self, address):
        self.gdb.execute("break *%s" % address)

    def finish(self):
        self.gdb.execute('finish')

    def step(self):
        self.gdb.execute('stepi')

    def resume(self):
        self.gdb.execute('continue')

    def nexti(self):
        self.gdb.execute('nexti')

    def get_instruction(self):
        return self.gdb.execute('x/i $rip', to_string=True)

`get_event()` is used to assign events based which can be either a `call` which represents a new frame instantiation, `line` event which represents a frame update as well as `return` which represent when a frame does not more exist in the current context.

In [ ]:
class BinaryDebugger(BinaryDebugger):
    def get_event(self, frame):
        fname = frame.function
        if fname not in self.frames:
            self.frames.append(fname)
            return 'call'
        elif fname == self.frames[-1]:
            return 'line'
        else:
            self.frames.pop()
            return 'return'

In order to debug stripped binaries, the first thing to consider is looking for the entry point address to the program. The function `get_entry_point()` is used to extract the entry point address to the program.

In [ ]:
class BinaryDebugger(BinaryDebugger):
    def get_entry_point(self):
        self.start_program(self.inp, self.binary)
        self.run()
        info_file = self.gdb.execute('info file', to_string=True)
        entry_address = ''
        for line in info_file.splitlines():
            if 'Entry point' in line:
                entry_address = line.split(':')[1]
                break
        return entry_address

Once the entry point address has been gotten, we then need to locate the address to the function `main()` which istypically where the program starts its execution from. To get this, we used the function `get_main()` to derive the main address that points to this function.

In [ ]:
class BinaryDebugger(BinaryDebugger):
    def get_main(self):
        self.gdb.execute('set confirm off')
        entry_addr = self.get_entry_point()
        self.break_at(entry_addr)
        self.run()

        instructions = []
        while True:
            next_i = self.get_instruction()
            if CALL in next_i:
                break
            instructions.append(next_i)
            self.step()

        instr = instructions[-1].split()
        if len(instr) == 6: s = instr[3]
        else: s = instr[4]

        reg = s[-3:]
        main_addr = gdb.execute('p/x $%s' % reg, to_string=True)
        main_addr = main_addr.partition("= ")

        return main_addr[-1]

In order not to go out of context (i.e stepping into represents standard libraries instructions), function `get_address_range()` is used to get the address range of the set of instruction which our programs contains. This would serve as a guiding path when stepping through instructions.

In [ ]:
class BinaryDebugger(BinaryDebugger):
    def get_address_range(self):
        start_addr = None
        end_addr = None
        mappings = self.gdb.execute('info proc mappings', to_string=True)

        for i, line in enumerate(mappings.splitlines()):
            if i == 4:
                start_addr = line.split()[0]
            elif i == 6:
                end_addr = line.split()[1]
        return (start_addr, end_addr)

The function `in_scope()` uses the address range to make the debugger aware of when it is on the right path or when it encounters a wrong path.

In [ ]:
class BinaryDebugger(BinaryDebugger):
    def in_scope(self, instr, start, end):
        instr = instr.split()
        instr.pop(0)

        current_addr = instr[0].strip(':')
        hex_val = int(current_addr, 16)
        if hex_val in range(int(start, 16), int(end, 16)):
            return True
        else:
            return False

The function `event_loop()` encompasses and makes use of all the logic defined above for the debugger,in order to step through all the instruction set in the right path to produce a trace which would be use to derive our desired grammar.

In [ ]:
class BinaryDebugger(BinaryDebugger):
    def event_loop(self):
        main = self.get_main()
        self.break_at(main)
        self.resume()

        start, end = self.get_address_range()
        nexti = ''
        current_frame = None
        while True:
            try:
                nexti = self.get_instruction()
                if self.in_scope(nexti, start, end):
                    h = Instruction(nexti)
                    if h.instr_type == CALL:
                        if h.symbol_name not in self.methods and\
                            h.symbol_name != None:
                            self.nexti()
                            continue
                        else:
                            self.step()
                            current_frame = Frame(h, self.inp)
                    else:
                        self.step()
                        if current_frame != None:
                            current_frame.update(h)
                        else:
                            continue
                    event = self.get_event(current_frame)
                    self.tracer.traceit(current_frame, event, None)
                else:
                    self.finish()
            except gdb.error:
                break

### Input Stack

We update the `InputStack` class in order to account for fragments with single length.

In [ ]:
from GrammarMiner import InputStack, ScopedVars, ScopeTracker

In [ ]:
class BinaryInputStack(InputStack):
    def __init__(self, i, fragment_len):
        super().__init__(i, fragment_len)

In [ ]:
class ScopedVars(ScopedVars):
    def create_call_stack(self, i):
        return BinaryInputStack(i, fragment_len=1)

In [ ]:
class ScopeTracker(ScopeTracker):
    def create_assignments(self, *args):
        return ScopedVars(*args)

### Binary Scope TreeMiner


In [ ]:
from GrammarMiner import ScopeTreeMiner, to_nonterminal, is_nonterminal

The class `BinaryScopeTreeMiner` is same as the `ScopeTreeMiner`, the only difference is that since line numbers are represented as string address, we change the format from `%d` to `%s`.

In [ ]:
class BinaryScopeTreeMiner(ScopeTreeMiner):
    def nt_var(self, key):
        method, seq, var, lno = key
        return to_nonterminal("%s@%s:%s" % (method, lno, var))

In [ ]:
class BinaryScopeTreeMiner(BinaryScopeTreeMiner):
    def string_part_of_value(self, part, value):
        return (part in value)

### Binary Scope Grammar Miner

The class `BinaryScopedGrammarMiner` is the same as the class `ScopedGrammarMiner` with the only difference being that we now use our newly updated`BinaryScopeTreeMiner`.

In [ ]:
from GrammarMiner import ScopedGrammarMiner, START_SYMBOL

In [ ]:
class BinaryScopedGrammarMiner(ScopedGrammarMiner):
    def create_tree_miner(self, *args):
        return BinaryScopeTreeMiner(*args)

    def create_tracker(self, *args):
        return ScopeTracker(*args)

## Method Generalization

The idea here is that, sometimes one finds that our central assumption -- that a fragment consumed by a function can be replaced by another fragment consumed by the same function elsewhere -- doesn't hold. This can be seen in functions that take an additional argument to specify what it should match. In such cases, we want to try and find out how to distinguish between these function invocations. 

Caching and book keeping variables.

In [ ]:
import json
NODE_REGISTER = {}
EXEC_MAP = {}

In [ ]:
def reset_generalizer():
    global EXEC_MAP
    global NODE_REGISTER
    EXEC_MAP.clear()
    NODE_REGISTER.clear()

In [ ]:
def format_mined_tree(tree, executable, inp):
    return {'tree': tree, 'original': executable, 'arg': inp}

In [ ]:
def is_nt(v):
    return len(v) > 1 and (v[0], v[-1]) == ('<', '>')

A small library function to convert from tuple to lists so that we can modify a tree.

In [ ]:
def to_modifiable(derivation_tree):
    node, children, *rest = derivation_tree
    return [node, [to_modifiable(c) for c in children], *rest]

In [ ]:
def tree_to_str(tree):  # Non recursive
    expanded = []
    to_expand = [tree]
    while to_expand:
        (key, children, *rest), *to_expand = to_expand
        if is_nt(key):
            to_expand = children + to_expand
        else:
            assert not children
            expanded.append(key)
    return ''.join(expanded)

The `get_ref()` takes a `node` datastructure and searches for `node_name`. It returns the first instance found. This allows us to easily swap nodes.

In [ ]:
def get_ref(node, node_name):
    name, children, *rest = node
    if name == node_name:
        return node
    for child in children:
        res = get_ref(child, node_name)
        if res is not None: return res
    return None

 The `replace_nodes()` function try to replace the contents of the first node with the _contents_ of the second (That is, the tree that has these nodes will automatically be modified), collect the produced string from the tree, and reset any changes. The arguments are tuples with the following format: (node, file_name, tree)

In [ ]:
def deep_copy(t):  # Python deepcopy is a bit buggy
    v = json.dumps(t)
    return json.loads(v)

In [ ]:
# replace the given node in a2 by the node in a1
def replace_nodes(a2, a1):
    node2, _, t2 = a2
    node1, _, t1 = a1
    str2_old = tree_to_str(t2)

    # first change the name of the node, then copy the tree.
    tmpl_name = '___bminer___'
    old_name = node2[0]
    node2[0] = tmpl_name
    t2_new = deep_copy(t2)
    node2[0] = old_name

    # now find the reference to tmpl_name in t2_new
    node2 = get_ref(t2_new, tmpl_name)
    node2.clear()
    for n in node1:
        node2.append(n)
    str2_new = tree_to_str(t2_new)
    assert str2_old != str2_new
    return t2_new

Can a given node be replaced with another? The idea is, given two nodes (possibly from two trees), can the first node be replaced by the second, and still result in a valid string?

In [ ]:
def is_a_replaceable_with_b(a1, a2, module):
    n1, f1, t1 = a1
    n2, f2, t2 = a2
    if tree_to_str(n1) == tree_to_str(n2): return True
    t_x = replace_nodes(a1, (('XXXX', []), None, t2))
    x = tree_to_str(t_x)
    updated_tree = replace_nodes(a1, a2)
    o = tree_to_str(t1)
    v = check(o, x, n1[0], updated_tree, module, tree_to_str(a1[0]), tree_to_str(a2[0]))
    return False

In [ ]:
def is_compatible(a1, a2, module):
    t1 = is_a_replaceable_with_b(a1, a2, module)
    if not t1: return False
    t2 = is_a_replaceable_with_b(a2, a1, module)
    return t2

There are fundamentally, two kinds of nodes. The first kind of node is a method node, that correspond to a method call. The second is a node that corresponds to a pseudo-method -- that is, a node that represents a loop or a conditional. Below are the predicates that identify such methods, parses, and reconstructs such nodes.

In [ ]:
def is_node_method(node):
    node_name = node[0]
    if (node_name[0], node_name[-1]) != ('<', '>'): return False
    return not is_node_pseudo(node)

In [ ]:
def is_node_pseudo(node):
    node_name = node[0]
    if (node_name[0], node_name[-1]) != ('<', '>'): return False
    if ':if_' in node_name: return True
    if ':while_' in node_name: return True
    return False

In [ ]:
def parse_pseudo_name(node_name):
    assert (node_name[0], node_name[-1]) == ('<', '>')
    return decode_name(node_name[1:-1])

In [ ]:
def parse_method_name(mname):
    assert (mname[0], mname[-1]) == ('<', '>')
    name = mname[1:-1]
    if '.' in name:
        nname, my_id = name.split('.')
        return nname, my_id
    else:
        return name, '0'

In [ ]:
def decode_name(node_name_stack):
    node_name, mstack = node_name_stack.split('#')
    method_stack = json.loads(mstack)
    method_ctrl_alt_name, can_empty = node_name.split(' ')
    method, ctrl_cid_altid = method_ctrl_alt_name.split(':')
    ctrl, cid_altid = ctrl_cid_altid.split('_')
    assert ctrl in {'while', 'if'}
    cid, altid = cid_altid.split(',')

    if 'while' == ctrl:
        assert altid == '0'
    return method, ctrl, int(cid), altid, can_empty, method_stack

In [ ]:
def unparse_pseudo_name(method, ctrl, ctrl_id, alt_num, can_empty, cstack):
    return "<%s>" % encode_name(method, ctrl, ctrl_id, alt_num, can_empty, cstack)

def unparse_method_name(mname, my_id):
    return '<%s.%s>' % (mname, my_id)

def encode_name(method, ctrl, ctrl_id, alt_num, can_empty, stack):
    assert ctrl in {'while', 'if'}
    return '%s:%s_%s,%s %s#%s' % (method, ctrl, ctrl_id, alt_num, can_empty, json.dumps(stack))

The `check()` function invokes the given subject call with the previously defined `check.py` wrapper, logs and returns the result of the call.

**TODO**: What we really want to do, is to generate a new updated tree first after doing the tree surgery. Then, convert this tree to a parenthesized tree by simply doing `tree_to_string` with additional `{}` (or other open/close symbols that does not conflict with the input) attached when joining the nodes. That is,

In [ ]:
def tree_to_pstr(tree, op_='', _cl=''):
    expanded = []
    to_expand = [tree]
    while to_expand:
        (key, children, *_), *to_expand = to_expand
        if is_nt(key):
            expanded.append(op_)
            to_expand = children + [(_cl, [])] + to_expand
        else:
            assert not children
            expanded.append(key)
    return ''.join(expanded)

With this, when using `tree_to_string(my_tree, '{', '}')`, We will get a string that represents how the original string was parsed. For example `1+2+3` may be represented as `{{1+2}+3}`.

Next, we need to run the non-parenthesized string resulting from the tree surgery through the program, and collect the resulting tree. Again, convert this tree to the parentesized version, and compare equality.

With this, we can ensure that our tree nodes are correctly compatible, and secondly, we can ignore the return code.

In [ ]:
def save_inp(inp):
    with open(f'subjects/inptxt', 'w+') as f:
        print(inp, file=f)

In [ ]:
import subprocess
def check(o, x, e, ut, module, sa1, sa2):
    s = tree_to_str(ut)
    if s in EXEC_MAP: return EXEC_MAP[s]
    updated_ps = tree_to_pstr(ut, op_='{', _cl='}')
    
    subprocess.call(['gcc', '-g', module])
    #!rm gdb.txt
    #!rm gdbtrace
    #!rm function_names
    
    traces = []
    try:
        save_inp(s)
        arg = '\'py arg0="%s"\'' % s
        argfiles = '\'py files="%s"\'' % 'a.out'
        !gdb --batch-silent -ex {argfiles} -x miner.py
        with open(f'gdbtrace', 'rb') as f:
            traces.append((s, jsonpickle.decode(f.read())))

        tracker = ScopeTracker(traces[0][0], traces[0][1])
        parsed_tree = BinaryScopeTreeMiner(traces[0][0],
            tracker.my_assignments.defined_vars(formatted=False))
        
        parsed_ps = tree_to_pstr(parsed_tree.tree, op_='{', _cl='}')
        v = (parsed_ps == updated_ps)
    except:
        parsed_ps = 'ERROR'
        v = False

    EXEC_MAP[s] = v
    return v

We now want to collect all nodes of a particular kind together. `register_node()` correctly saves specific kinds of nodes separately as copies.

In [ ]:
def register_node(node, tree, executable, input_file):
    # we want to save a copy of the tree so we can modify it later. 
    node_name = node[0]
    template_name = '__BMINER__NODE__'
    node[0] = template_name
    new_tree = deep_copy(tree)
    node[0] = node_name
    new_node = get_ref(new_tree, template_name)
    new_node[0] = node_name
    if node_name not in NODE_REGISTER:
        NODE_REGISTER[node_name] = []
    new_elt = (new_node, new_tree, executable, input_file,
            {'inputstr': tree_to_str(new_tree), 'node':node, 'tree':tree})
    NODE_REGISTER[node_name].append(new_elt)
    return new_elt

In [ ]:
def collect_nodes_single(node, tree, executable, inputfile, seen):
    node_name, children, si = node
    elt = None
    if is_node_method(node):
        elt = register_node(node, tree, executable, inputfile)
        if node_name in seen:
            elt[4]['seen'] = seen[node_name]
        else:
            seen[node_name] = elt
    if len(children) == 1:
        collect_nodes_single(children[0], tree, executable, inputfile, seen)
    else:
        # no longer the single inheritance line.
        for child in children:
            collect_nodes(child, tree, executable, inputfile)

In [ ]:
def collect_nodes(node, tree, executable, inputfile):
    node_name, children, si = node
    elt = None
    if is_node_method(node):
        elt = register_node(node, tree, executable, inputfile)
    if len(children) == 1:
        collect_nodes_single(children[0], tree, executable, inputfile, {node_name: elt})
    else:
        for child in children:
            collect_nodes(child, tree, executable, inputfile)

When looking to see which methods are swappable, the idea is to choose a small sample set for a given node name, and check the current node against that sample set (swap both ways, and check the validity). The different validity patterns we get are marked as different kinds of nodes.

Here, we identify the buckets based on one to one compatibility. Unfortunately, there is a problem here. Essentially, we assume that if `a` is compatible with `b`, and `b` is compatible with `c`, then `a` is compatible with `c`. However, this may not be true in all cases. See the limitations for instances when this assumption is invalidated. At this point, we have several options. The first is to do an $n \times n$ comparison of all items in the bucket, in which case, we will have the true compatibility but with high computational cost. The next alternative is to choose a node in one bucket, and do the bucketing procedure again with the items in the particular bucket. This produces one more bit of information, and one can continue this prodcedure for larger and larger number of bits. One may also choose a statistical sample of $k$ items in the bucket, and go for a comparison only between $n \times k$ items.

At this point, we choose the fastest option, which gets us a reasonable accuracy. We use a single level classification.

In [ ]:
def identify_buckets(node_name):
    all_elts = NODE_REGISTER[node_name]
    # remove the duplicate nodes
    elts = [e for e in all_elts if 'seen' not in e[4]]
    seen_elts = [e for e in all_elts if 'seen' in e[4]]
    first, *rest = elts
    first[4]['pattern'] = 0
    buckets = [first]
    for enode in rest:
        node0, tree0, executable0, inputfile0, _info0 = enode
        a0 = node0, inputfile0, tree0
        compatible = None
        for bi, bnode in enumerate(buckets):
            node1, tree1, executable1, inputfile1, _info1 = bnode
            a1 = node1, inputfile1, tree1
            result = is_compatible(a0, a1, executable0)
            if result:
                compatible = bi
                enode[4]['pattern'] = bi
                break
        if compatible is None:
            enode[4]['pattern'] = len(buckets)
            buckets.append(enode)

    for e in seen_elts:
        e_seen = e[4]['seen']
        e_seen_pattern = e_seen[4]['pattern']
        e[4]['pattern'] = e_seen_pattern
    return {i: i for i, b in enumerate(buckets)}

Once we identify that a node belongs to a particular pattern identifier, we update all the pseudo-methods belonging to that node. These can be found by simply traversiing the tree until the next method is found.

In [ ]:
def update_method_stack(node, old_name, new_name):
    nname, children, *rest = node
    if not (':if_' in nname or ':while_' in nname):
        return
    method, ctrl, cname, num, can_empty, cstack = parse_pseudo_name(nname)
    assert method == old_name, "%s != %s" % (method, old_name)
    name = unparse_pseudo_name(new_name, ctrl, cname, num, can_empty, cstack)
    #assert '?' not in name
    node[0] = name
    for c in node[1]:
        update_method_stack(c, old_name, new_name)

In [ ]:
def update_method_name(k_m, my_id):
    # fixup k_m with what is in my_id
    original = k_m[0]
    method, old_id = parse_method_name(original)
    name = unparse_method_name(method, my_id)
    k_m[0] = name

    for c in k_m[1]:
        update_method_stack(c, original[1:-1], name[1:-1])

    return name, k_m

In [ ]:
def update_original_method_names(node_name):
    registered_xnodes = NODE_REGISTER[node_name]
    for xnode in registered_xnodes:
        # name it according to its pattern
        nodeX, treeX, executableX, inputfileX, infoX = xnode
        pattern = infoX['pattern']
        update_method_name(infoX['node'], pattern)

The idea is to first collect and register all nodes by their names.
Next, we sample N of these, and use the pattern of matches

In [ ]:
def generalize_method_trees(jtrees, log=False):
    my_trees = []
    for i, j in enumerate(jtrees):
        tree = to_modifiable(j['tree'])  # The tree ds.
        executable = j['original']
        inputfile = j['arg']
        # we skip START
        node_name, children, *rest = tree
        assert node_name == START_SYMBOL
        for child in children:
            collect_nodes(tree, tree, executable, inputfile)
        my_trees.append({'tree':tree, 'original': executable, 'arg': inputfile})
        
    for k in NODE_REGISTER:
        identify_buckets(k)
    
    # finally, update the original names.
    for k in NODE_REGISTER:
        if k == START_SYMBOL: continue
        update_original_method_names(k)
    return my_trees

## Using Debug Adapter Protocol

### DAP Debug Session

In [ ]:
class IDebugSession:
    def attach(self, request):
        raise NotImplementedError()

    def launchRequest(self, request):
        raise NotImplementedError()

    def initializeRequest(self, request):
        raise NotImplementedError()

In [ ]:
class DAPDebugSession(IDebugSession):
    def __init__(self):
        self.d = None

    def launchRequest(self, request):
        arguments = request['arguments']
        debugee = arguments['debugee']
        arg = arguments['arg']
        kwargs = arguments['kwarg']

        self.d = BinaryDebugger(gdb, debugee, arg, **kwargs)
        self.d.event_loop()
        trace = self.d.tracer.trace
        return trace

In [ ]:
class DAPDebugSession(DAPDebugSession):
    def initializeRequest(self, request):
        payload = {
            'supportsLogPoints': True,
            'supportsSetVariable': True
        }
        arguments = request['arguments']
        self.supportsRunInTerminalRequest = arguments[
            'supportsRunInTerminalRequest']
        return payload

### Debug Server

In [ ]:
import json
import socket
import sys

In [ ]:
class BinaryDebugServer:
    def __init__(self, host, port):
        self.EVENT = {
            'type': 'event',
            'body': {},
            'event': '',
            'seq': 0,
            'type': ''
        }
        self.RESPONSE = {
            'type': 'response',
            'success': False,
            'seq': 0,
            'request_seq': 0,
            'body': {},
            'command': '',
            'message': ''
        }
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind((host, port))
        self.socket.listen(20)
        self.debugSession = None

In [ ]:
class BinaryDebugServer(BinaryDebugServer):
    def sendResponse(self, conn, message):
        s_msg = json.dumps(message)
        response = 'Content-Length: {0}\r\n\r\n{1}'.format(len(s_msg), s_msg)
        response = bytes(response, 'utf-8')
        conn.send(response)

In [ ]:
class BinaryDebugServer(BinaryDebugServer):
    def parse_all_messages(self, msg_str):
        split_delimiter = 'Content-Length: '
        msg_list = msg_str.split(split_delimiter)
        if (msg_list[0] == ''):
            msg_list = msg_list[1:]
        parsed_msg_list = [msg.split('\r\n\r\n') for msg in msg_list]
        for index in range(len(parsed_msg_list)):
            parsed_msg_list[index][
                0] = split_delimiter + parsed_msg_list[index][0]
        return parsed_msg_list

In [ ]:
class BinaryDebugServer(BinaryDebugServer):
    def server_loop(self):
        while True:
            conn, _ = self.socket.accept()
            data = conn.recv(1024).decode()
            if not data:
                continue

            self.debugSession = DAPDebugSession()
            msgs = self.parse_all_messages(data)
            payload = msgs[0][1]
            payload = json.loads(payload)

            if payload['command'] == 'initialize':
                body = self.debugSession.initializeRequest(payload)
                res = self.RESPONSE
                res['body'] = body

                res['command'] = 'initialize'
                res['success'] = 'True'
                self.sendResponse(conn, res)

                res1 = self.EVENT
                res1['event'] = 'initialized'
                self.sendResponse(conn, res1)

            elif payload['command'] == 'launch':
                trace = self.debugSession.launchRequest(payload)
                fileName = 'gdbtrace'

                with open(fileName, 'w+') as f:
                    print(jsonpickle.encode(trace), file=f)
                self.sendResponse(conn, fileName)

            elif payload['command'] == 'functionNames':
                fn_names = self.debugSession.functionNameRequest(payload)
                with open('function_names', 'w+') as f:
                    print(jsonpickle.encode(fn_names), file=f)

In [ ]:
server_head = """\
import sys
sys.path.extend([%s])
sys.path.append('.')
import matplotlib.pyplot
matplotlib.pyplot._IP_REGISTERED = True # Hack
import fuzzingbook_utils
from GrammarMiner import GrammarMiner, Context, Tracer, Coverage, ScopedGrammarMiner, AssignmentVars, readable, flatten
import jsonpickle
import os, subprocess
import gdb
import re
import json
import socket
""" % (', '.join("'%s'" % str(i) for i in sys.path if i))
debugger_src = fuzzingbook_utils.extract_class_definition(Debugger)
context_src = fuzzingbook_utils.extract_class_definition(GDBContext)
gdbtracer_src = fuzzingbook_utils.extract_class_definition(GDBTracer)
varextractor_src = fuzzingbook_utils.extract_class_definition(VarExtractor)
gdbdebugger_src = fuzzingbook_utils.extract_class_definition(GDBDebugger)

instr_src = fuzzingbook_utils.extract_class_definition(Instruction)
frame_src = fuzzingbook_utils.extract_class_definition(Frame)
binary_cxt_src = fuzzingbook_utils.extract_class_definition(BinaryContext)
b_extractor_src = fuzzingbook_utils.extract_class_definition(BinaryVarExtractor)
binary_tracer_src = fuzzingbook_utils.extract_class_definition(BinaryTracer)
stripped_debugger_src = fuzzingbook_utils.extract_class_definition(
    BinaryDebugger)

IDebugSession_src = fuzzingbook_utils.extract_class_definition(IDebugSession)
DAPDebugSession_src = fuzzingbook_utils.extract_class_definition(DAPDebugSession)
BinaryDebugServer_src = fuzzingbook_utils.extract_class_definition(BinaryDebugServer)

helper = """
def recover_fnames(inp, debugee):
    fn_dict = {}
    fn_names = []
    gdb.execute("set args '%s'" % inp)
    gdb.execute("file %s" % debugee)
    gdb.execute('set confirm off')

    all_defined_functions = gdb.execute('info functions', to_string=True)
    lines = all_defined_functions.splitlines()[3:]
    for line in lines:
        if line:
            line = line.split()
            line = line[2:]
            if '(' in line[0]:
                l = line[0].split('(')
                fn_names.append(l[0].strip('*'))
            else:
                l = line[1].split('(')
                fn_names.append(l[0].strip('*'))
        else:
            break
    
    gdb.execute('run')
    for k in fn_names:
        try:
            s = gdb.execute('info address %s' % k, to_string=True)
            addr = s.split()[-1].strip('.')
            fn_dict[addr] = k
        except gdb.error:
            continue
    
    return fn_dict
"""

server_tail = """
b = BinaryDebugServer('', 8541)
b.server_loop()
"""
server_src = '\n'.join([
    server_head, debugger_src, context_src, gdbtracer_src, varextractor_src,
    gdbdebugger_src, global_declarations, helper_methods, instr_src, frame_src, binary_cxt_src,b_extractor_src, binary_tracer_src,
    stripped_debugger_src, helper, IDebugSession_src, DAPDebugSession_src, BinaryDebugServer_src, server_tail
])

**Attention**: This file can should be executed on a separate terminal using the command 
`gdb --batch-silent -x dapServer.py`


In [ ]:
with open('dapServer.py', 'w+') as f:
    print(server_src, file=f)

### Client Side

#### Setting up Connection

In [ ]:
from pprint import pprint                                       
import time   
import random

In [ ]:
class connectionClass:
    def __init__(self, hostname='', port=8541, timeout=5, sock=None):
        self.msg_id = random.randint(1,500)
        self.connection_no = 1
        self.mysocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.mysocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.port = port
        self.hostname = hostname
        self.mysocket.settimeout(timeout)
        self.connect()

In [ ]:
class connectionClass(connectionClass):
    def connect(self):
        self.mysocket.connect((self.hostname, self.port))
        self.connection_no += 1
        time.sleep(1)
        #the debug server responds to successful connection
        self.recvMessage("connection no {} ".format(self.connection_no))

In [ ]:
class connectionClass(connectionClass):
    def parse_all_messages(self, msg_bytes):
        msg_str = msg_bytes.decode('utf-8')
        split_delimiter = 'Content-Length: '
        msg_list = msg_str.split(split_delimiter)
        if (msg_list[0] == ''):
            msg_list = msg_list[1:]
        parsed_msg_list = [msg.split('\r\n\r\n') for msg in msg_list]
        for index in range(len(parsed_msg_list)):
            parsed_msg_list[index][
                0] = split_delimiter + parsed_msg_list[index][0]
        return parsed_msg_list

In [ ]:
class connectionClass(connectionClass):
    def recvMessage(self, func_name):
        sock = self.mysocket
        try:
            recv_data = sock.recv(1024 * 1024)
        except socket.timeout:
            print("Timeout. Nothing was received at", func_name)
            return
        list_of_msgs = self.parse_all_messages(recv_data)
        return list_of_msgs

In [ ]:
class connectionClass(connectionClass):
    def sendMessage(self, msg):
        sock = self.mysocket
        s_msg = json.dumps(msg)
        data = 'Content-Length: {0}\r\n\r\n{1}'.format(len(s_msg), s_msg)
        data = bytes(data, 'utf-8')
        sock.send(data)
        time.sleep(1)

In [ ]:
class connectionClass(connectionClass):
    def doRequest(self, msg):
        this_id = self.msg_id
        self.msg_id += 1
        msg['seq'] = this_id
        msg['type'] = 'request'
        self.sendMessage(msg)

In [ ]:
class connectionClass(connectionClass):
    def close_connection(self):
        self.recvMessage("Cleanup")
        self.mysocket.close()

    def reset_connection(self):
        self.close_connection()
        self.mysocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.connect()

#### Binary Client

In [ ]:
class BinaryClient:
    def __init__(self, hostname='', port=8541, timeout=5):
        self.connection = connectionClass(hostname, port, timeout)

In [ ]:
class BinaryClient(BinaryClient):
    def launch(self, seed, dbgfile, **kwargs):
        self.connection.doRequest({
            'command': 'launch',
            'type': 'request',
            'arguments': {
                'noDebug': 'False',
                '__restart': '',
                'debugee': dbgfile,
                'arg': seed,
                'kwarg': kwargs
            },
        })
        return self.connection.recvMessage("launch")

In [ ]:
class BinaryClient(BinaryClient):
    def initialize(self, clientID=None):
        if clientID is None:
            clientID = self.__class__.__name__
        self.connection.doRequest({
            'command': 'initialize',
            'arguments': {
                'adapterID': 'pydevd',
                'clientID': clientID,
            },
        })
        self.connection.recvMessage("initialize")

## Recovering Grammar

In [ ]:
def fit_tree(derivation_tree):
    node, children, *rest = derivation_tree
    return tuple([node, [fit_tree(c) for c in children], *rest])

In [ ]:
from IPython.display import Image

def zoom(v, zoom=True):
    # return v directly if you do not want to zoom out.
    if zoom:
        return Image(v.render(format='png'))
    return v

We updated the method `update_grammar` in order to make use method generalization on the mined trees.

In [ ]:
class BinaryScopedGrammarMiner(BinaryScopedGrammarMiner):
    def update_grammar(self, inputstr, trace, executable):
        at = self.create_tracker(inputstr, trace)
        dt = self.create_tree_miner(inputstr, at.my_assignments.defined_vars(formatted=False))
        
        tree = [format_mined_tree(dt.tree, executable, inputstr)]
        generalized_tree = generalize_method_trees(tree)
        new_tree = fit_tree(tuple(generalized_tree[0]['tree']))
        
        #v = display_tree(new_tree)
        self.add_tree(new_tree)
        #zoom(v)
        return self.grammar

In [ ]:
import itertools
from GrammarMiner import display_tree, lr_graph

In [ ]:
class BinaryScopedGrammarMiner(BinaryScopedGrammarMiner):
    def add_tree(self, t):
        t_grammar = self.tree_to_grammar(t)
        self.grammar = {
            key: self.grammar.get(key, []) + t_grammar.get(key, [])
            for key in itertools.chain(self.grammar.keys(), t_grammar.keys())
        }

# Evaluation

## Recover Grammar

In [ ]:
def recover_grammar(inps, src, executable):
    traces = []
    miner = BinaryScopedGrammarMiner()
    
    for c, inp in enumerate(inps):
        save_inp(inp)
        arg = '\'py arg0="%s"\'' % inp
        argfiles = '\'py files="%s"\'' % src
        print(arg)
        !gdb --batch-silent -ex {argfiles} -x miner.py
        with open(f'gdbtrace', 'rb') as f:
            traces.append((inp, jsonpickle.decode(f.read())))
        #!cp gdbtrace gdbtrace.{c} 
    
    
    for inp, trace in traces:
        reset_generalizer()
        miner.update_grammar(inp, trace, executable)
    return (readable(miner.clean_grammar()))

## Putting all together

In [ ]:
tracer_head = """\
import sys
sys.path.extend([%s])
sys.path.append('.')
import matplotlib.pyplot
matplotlib.pyplot._IP_REGISTERED = True # Hack
import fuzzingbook_utils
from GrammarMiner import GrammarMiner, Context, Tracer, Coverage, ScopedGrammarMiner, AssignmentVars, readable, flatten
import jsonpickle
import os, subprocess
import gdb
import re
import json                                                                     
import socket                                                                   
from pprint import pprint                                       
import time   
import random
""" % (', '.join("'%s'" % str(i) for i in sys.path if i))
connection_src = fuzzingbook_utils.extract_class_definition(connectionClass)
client_src = fuzzingbook_utils.extract_class_definition(BinaryClient)

tracer_tail = """
file_name = 'gdbtrace'
binary = 'a.out'
def recover_fnames(inp, debugee):
    fn_dict = {}
    fn_names = []
    gdb.execute("set args '%s'" % inp)
    gdb.execute("file %s" % debugee)
    gdb.execute('set confirm off')

    all_defined_functions = gdb.execute('info functions', to_string=True)
    lines = all_defined_functions.splitlines()[3:]
    for line in lines:
        if line:
            line = line.split()
            line = line[2:]
            if '(' in line[0]:
                l = line[0].split('(')
                fn_names.append(l[0].strip('*'))
            else:
                l = line[1].split('(')
                fn_names.append(l[0].strip('*'))
        else:
            break
    
    gdb.execute('run')
    for k in fn_names:
        try:
            s = gdb.execute('info address %s' % k, to_string=True)
            addr = s.split()[-1].strip('.')
            fn_dict[addr] = k
        except gdb.error:
            continue
    
    with open('function_names', 'w+') as f:
        print(jsonpickle.encode(fn_dict), file=f)

def request_loop(gdb, f, inp, **kwargs):
    client = BinaryClient()
    trace = client.launch(inp, f, **kwargs)
    client.connection.close_connection()
    
def recover_trace(f, inp, **kwargs):
    request_loop(gdb, f, inp, **kwargs)

arg_0 = None
with open(f'subjects/inptxt', 'r+') as f:
    arg_0 = f.read().replace('\\n', '')

if not os.path.exists('function_names'):
    recover_fnames(str(arg_0), binary)

subprocess.call(['strip', '-s', binary])
recover_trace(binary, str(arg_0), files=[binary])
"""
tracer_src = '\n'.join([
    tracer_head, connection_src, client_src, tracer_tail
])

In [ ]:
with open('miner.py', 'w+') as f:
    print(tracer_src, file=f)

## Subjects

#### CGIDecode

In [ ]:
cgidecode_samples = [
    "Hello%2c+world%21",
    'Send+mail+to+me%40fuzzingbook.org',
    'name=Fred&class=101&math=2+%2B+2+%3D+4&',
    'name=Fred&status=good&status=happy&',
    'http://target/getdata.php?data=%3cscript%20src=%22http%3a%2f%2f',
    'www.badplace.com%2fnasty.js%22%3e%3c%2fscript%3e',
    'http://target/login.asp?userid=bob%27%3b%20update%20logintable%20set%20passwd%3d%270wn3d%27%3b--%00',
    'Colon%20%3A%20Hash%20%23%20Percent%20%25',
    'O%21nP%22BG%23JI%24Tw%25mJ%26bB%27xX%28zy%29Aj%2aZ',
    'E%2bNp%2cRP%2dVN%2eyV%2ftW%2AIJ%2BAe%2CkM%2DKf%2EB',
    'W%2FAo%3azF%3blw%3ctY%3dqy%3eLm%3fCS%3AyB%3BHq%3Ck',
    'y%3DZM%3EVH%3FRx%40gG%5bhh%5cjn%5dOD%5eDR%5fcu%5Bm',
    'b%5CJm%5Drl%5Ezn%5FKe%60hQ%7bBj%7chf%7dmB%7eyc%7Bp',
    'w%7CWd%7DCG%7Ec',
    'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
    '1234567890',
    '-./_~'
]

In [ ]:
!gcc -g -o a.out subjects/cgi_decode.c
!rm gdb.txt
!rm gdbtrace
!rm function_names

In [ ]:
grammar = recover_grammar([cgidecode_samples[0]], 'a.out', 'subjects/cgi_decode.c')
grammar

#### URLParse

In [ ]:
from GrammarMiner import URLS_X, START_SYMBOL

In [ ]:
!gcc -g -o a.out subjects/url_parse.c
!rm gdb.txt
!rm gdbtrace
!rm function_names

In [ ]:
grammar = recover_grammar(URLS_X, 'a.out', 'subjects/url_parse.c')
grammar

In [ ]:
from GrammarMiner import GrammarFuzzer, URLS_X, VEHICLES
x = GrammarFuzzer(grammar)

for i in range(10):
    print(x.fuzz())

#### Calculator

In [ ]:
calc_samples = [i.strip() for i in '''
33/44+2
'''.strip().split('\n') if i.strip()]

In [ ]:
#!gcc -g subjects/calc_parse.c
#!rm gdb.txt
#!rm gdbtrace
#!rm function_names

In [ ]:
#calc_grammar = recover_grammar(calc_samples, 'a.out', 'subjects/calc_parse.c')
#calc_grammar

In [ ]:
from GrammarMiner import GrammarFuzzer, URLS_X
x = GrammarFuzzer(calc_grammar)

for i in range(10):
    print(x.fuzz())

#### JSON

In [ ]:
json_samples = [i.strip().replace('\n', ' ') for i in [
'''
{"emptya":[], "emptyh":{}, "emptystr":"", "null":null}
''']]

In [ ]:
!gcc -g -o a.out subjects/json.c
#!rm a.out
!rm gdb.txt
!rm gdbtrace
!rm function_names

In [ ]:
json_grammar = recover_grammar(json_samples, 'a.out', 'subjects/json.c')
json_grammar

In [ ]:
from GrammarMiner import GrammarFuzzer, URLS_X
x = GrammarFuzzer(json_grammar)

for i in range(10):
    print(x.fuzz())

#### CSV

In [ ]:
csv_samples = [i.strip() for i in '''
1997,van,Ford,35E
2828,Mecuy,Couga
'''.strip().split('\n') if i.strip()]

In [ ]:
!gcc -g subjects/csvparser.c
!rm gdb.txt
!rm gdbtrace
!rm function_names

In [ ]:
csv_grammar = recover_grammar([csv_samples[0]], 'a.out', 'subjects/csvparser.c')
csv_grammar

#### INI

In [ ]:
ini_samples = [i.strip() for i in '''
a=b
[v]
'''.strip().split('\n') if i.strip()]

In [ ]:
!gcc -g subjects/ini.c
!rm gdb.txt
!rm gdbtrace
!rm function_names

In [ ]:
ini_grammar = recover_grammar(ini_samples, 'a.out', 'subjects/ini.c')
ini_grammar